### Scrape the Association of Banks Malaysia
We will now need to get a list of the Malaysian Banks. Thus scouring the internet for a bit, we find a website that list all the Banks in Malaysia with a **Commercial Banking** license. <br>
Line 1: URL of the Association of banks Malaysia <br>
Lines 3 - 6 : Creating the soup object to be parsed <br>
Lines 8 : All the data we are intereseted in is nested in the `<div class="d-none au-am-data">` tag <br>

In [1]:
from fake_useragent import UserAgent
ua = UserAgent()
random_header = ua.chrome
# print(random_header)

random_user_agent = {'User-Agent' : str(random_header)}

ModuleNotFoundError: No module named 'fake_useragent'

In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
my_url = 'https://www.abm.org.my/about-us/abm-members/'

abm_source = requests.get(url=f'{my_url}',\
               stream=True,\
               headers=random_user_agent).text  # => "<!DOCTYPE html><html><head>..."
abm_soup = BeautifulSoup(abm_source, 'html.parser')
#print(abm_soup.prettify())
malaysian_banks = abm_soup.find_all(name='div', attrs={'class': "d-none au-am-data"})
print(f'We have {len(malaysian_banks)} banks in Malaysia with a commercial banking license')

We have 26 banks in Malaysia with a commercial banking license


In [29]:
chime.themes

<function chime.themes() -> List[str]>

In [4]:
bank_names = []
hashtag_names = []
admirals_of_the_banks = []
top_titles = []
bank_websites = []


for each_bank in malaysian_banks:
    each_bank_name = each_bank.h5.text    
    each_hashtag = each_bank.get('data-bank')
    each_admiral = each_bank.find(name='div', attrs={'class':"au-am-value"}).text
    each_title = each_bank.find(name='div', attrs={'class':"au-am-label"}).text
    each_website = each_bank.find(name='a', attrs={'class':"au-am-web"}).get('href')                    
        
    bank_names.append(each_bank_name)
    hashtag_names.append(each_hashtag)
    admirals_of_the_banks.append(each_admiral)
    top_titles.append(each_title)
    bank_websites.append(each_website)

import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 1000)
import numpy as np
import matplotlib.pyplot as plt

robust_zip = list(zip(bank_names, hashtag_names, admirals_of_the_banks, top_titles, bank_websites))
banks_malaysia= pd.DataFrame(robust_zip, columns=['Bank_Names', 'Hashtag_Name', 'Top_Person', 'Top_Title', 'Bank_Website'])
banks_malaysia.shape

(26, 5)

In [5]:
from collections import defaultdict
hash_to_handle_map = {}
for idx, value in banks_malaysia['Hashtag_Name'].items():
    hash_to_handle_map.update({value : str()})


hash_to_handle_map['#affin_bank_berhad'] = '@AFFIN'
hash_to_handle_map.update({'#alliance_bank_malaysia_berhad' : '@AllianceBankMY',
                           '#ambank_(m)_berhad' : '@AmBankTV',
                           '#bangkok_bank_berhad' : '@bangkokbankchannel',
                           '#bank_of_america_malaysia_berhad' : '@BankofAmerica',
                           '#bank_of_china_(malaysia)_berhad' : '@BankofChinaManila',
                           '#bnp_paribas_malaysia_berhad' : '@labanquedunmondequichange',
                           '#boost_bank_berhad' : '@MyBoostApp',
                           '#china_construction_bank_(malaysia)_berhad': None,
                           '#cimb_bank_berhad' : '@CIMBGroupHoldingsBhd',
                           '#citibank_berhad' : '@Citi',
                           '#deutsche_bank_(malaysia)_berhad' : '@DeutscheBank',
                           '#gx_bank_berhad' : '@GXBank',
                           '#hong_leong_bank_berhad' : '@hongleongbankmy',
                           '#hsbc_bank_malaysia_berhad' : '@HSBC_MY',
                           '#industrial_and_commercial_bank_of_china_(malaysia)_berhad' : None,
                           '#j.p._morgan_chase_bank_berhad' : '@jpmorgan',
                           '#malayan_banking_berhad_(maybank)' : '@maybankvideos',
                           '#mizuho_bank_(malaysia)_berhad' : '@MizuhoAmericas',
                           '#mufg_bank_(malaysia)_berhad': '@MUFGBankChannel',
                           '#ocbc_bank_(malaysia)_berhad': '@channelocbc',
                           '#public_bank_berhad': '@PublicBankGroup',
                           '#rhb_bank_berhad': '@rhbgroup',
                           '#standard_chartered_bank_malaysia_berhad': '@standardchartered',
                           '#sumitomo_mitsui_banking_corporation_malaysia_berhad': '@smbcgroup',
                           '#united_overseas_bank_(malaysia)_bhd': '@uob'  
                          })

# Add the dict to the dataframe
banks_malaysia['Youtube_Handle'] = banks_malaysia['Hashtag_Name'].apply(lambda x: hash_to_handle_map.get(x, 'not found'))

In [6]:
# View the data
display(pd.concat([banks_malaysia.head(), banks_malaysia.tail()]))

,Bank_Names,Hashtag_Name,Top_Person,Top_Title,Bank_Website,Youtube_Handle
0,Affin Bank Berhad,#affin_bank_berhad,Datuk Wan Razly Abdullah Wan Ali,Group CEO,https://www.affingroup.com/affin-bank-berhad,@AFFIN
1,Alliance Bank Malaysia Berhad,#alliance_bank_malaysia_berhad,Kellee Kam Chee Khiong,Group CEO,https://www.alliancebank.com.my/,@AllianceBankMY
2,AmBank (M) Berhad,#ambank_(m)_berhad,Jamie Ling,Group CEO,https://www.ambankgroup.com/eng/Pages/home.aspx,@AmBankTV
3,Bangkok Bank Berhad,#bangkok_bank_berhad,Kanet Buranasin,CEO,http://www.bangkokbank.com.my/,@bangkokbankchannel
4,Bank of America Malaysia Berhad,#bank_of_america_malaysia_berhad,Gautam Padmakar Puntambekar,MD/Malaysia Country Head,https://www.bofaml.com/en-us/content/apac-malaysia.html,@BankofAmerica
21,Public Bank Berhad,#public_bank_berhad,Tan Sri Dato' Sri Dr Tay Ah Lek,MD/CEO,http://www.publicbankgroup.com/,@PublicBankGroup
22,RHB Bank Berhad,#rhb_bank_berhad,Dato' Mohd Rashid Mohamad,Group Managing Director / Group Chief Executive Officer,https://www.rhbgroup.com/,@rhbgroup
23,Standard Chartered Bank Malaysia Berhad,#standard_chartered_bank_malaysia_berhad,Mak Joon Nien,Managing Director & Chief Executive Officer,http://www.sc.com/my,@standardchartered
24,Sumitomo Mitsui Banking Corporation Malaysia Berhad,#sumitomo_mitsui_banking_corporation_malaysia_berhad,Atsuhide Shiojiri,President/CEO,https://www.smbc.co.jp/asia/malaysia/,@smbcgroup
25,United Overseas Bank (Malaysia) Bhd,#united_overseas_bank_(malaysia)_bhd,Ng Wei Wei,CEO,http://www.uob.com.my/,@uob


In [7]:
import pickle

In [8]:
## Write table 
with open('./Data/abm_banks_malaysia.pkl', 'wb') as f:
    pickle.dump(banks_malaysia, f)

In [9]:
## Read back data
# Takes a while to read the pickle
import pickle
# Reading the file - unpickling
with open('./Data/abm_banks_malaysia.pkl', 'rb') as fp:
    abm_banks = pickle.load(fp)

In [10]:
pd.concat([abm_banks.head(), abm_banks.tail()])

,Bank_Names,Hashtag_Name,Top_Person,Top_Title,Bank_Website,Youtube_Handle
0,Affin Bank Berhad,#affin_bank_berhad,Datuk Wan Razly Abdullah Wan Ali,Group CEO,https://www.affingroup.com/affin-bank-berhad,@AFFIN
1,Alliance Bank Malaysia Berhad,#alliance_bank_malaysia_berhad,Kellee Kam Chee Khiong,Group CEO,https://www.alliancebank.com.my/,@AllianceBankMY
2,AmBank (M) Berhad,#ambank_(m)_berhad,Jamie Ling,Group CEO,https://www.ambankgroup.com/eng/Pages/home.aspx,@AmBankTV
3,Bangkok Bank Berhad,#bangkok_bank_berhad,Kanet Buranasin,CEO,http://www.bangkokbank.com.my/,@bangkokbankchannel
4,Bank of America Malaysia Berhad,#bank_of_america_malaysia_berhad,Gautam Padmakar Puntambekar,MD/Malaysia Country Head,https://www.bofaml.com/en-us/content/apac-malaysia.html,@BankofAmerica
21,Public Bank Berhad,#public_bank_berhad,Tan Sri Dato' Sri Dr Tay Ah Lek,MD/CEO,http://www.publicbankgroup.com/,@PublicBankGroup
22,RHB Bank Berhad,#rhb_bank_berhad,Dato' Mohd Rashid Mohamad,Group Managing Director / Group Chief Executive Officer,https://www.rhbgroup.com/,@rhbgroup
23,Standard Chartered Bank Malaysia Berhad,#standard_chartered_bank_malaysia_berhad,Mak Joon Nien,Managing Director & Chief Executive Officer,http://www.sc.com/my,@standardchartered
24,Sumitomo Mitsui Banking Corporation Malaysia Berhad,#sumitomo_mitsui_banking_corporation_malaysia_berhad,Atsuhide Shiojiri,President/CEO,https://www.smbc.co.jp/asia/malaysia/,@smbcgroup
25,United Overseas Bank (Malaysia) Bhd,#united_overseas_bank_(malaysia)_bhd,Ng Wei Wei,CEO,http://www.uob.com.my/,@uob


# Test connection

## Requests and bs4

In [1]:
from fake_useragent import UserAgent
ua = UserAgent()
random_header = ua.chrome
# print(random_header)

random_user_agent = {'User-Agent' : str(random_header)}

import cloudscraper
import requests
from bs4 import BeautifulSoup

my_query='@bluestar9000'

my_url = f'https://www.youtube.com/{my_query}'
cloud_source = requests.get(url=f'{my_url}',\
               stream=True,\
               headers=random_user_agent).text  # => "<!DOCTYPE html><html><head>..."
cloud_soup = BeautifulSoup(cloud_source, 'html.parser')

my_channel_ID = cloud_soup.find(name='meta', attrs={'itemprop' : "identifier"})
print(f'The channel_name {my_query} has channel_ID : ', my_channel_ID.get('content'))

my_channel_ID = cloud_soup.find(name='meta', attrs={'itemprop' : "identifier"})

The channel_name @bluestar9000 has channel_ID :  UCkdWGOjnGGSY1PoDmKqiwSQ


## Selenium

In [84]:
import os
import time 
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import time 
from datetime import datetime

In [99]:
now = datetime.now()
now.strftime("%m-%d-%Y  %H:%M:%S")

'10-24-2025  16:47:22'

In [168]:
BIG_DICT = {}
service = Service("/usr/local/bin/chromedriver")
driver = webdriver.Chrome(service = service)
my_query='@bluestar9000'
my_url = f'https://www.youtube.com/{my_query}'
# Now that we have service, let's get the first url in our list
driver.get(url=my_url)
try:
    # Next we have to wait for the page to load
    WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.CLASS_NAME, "yt-truncated-text__absolute-button"))
    )
    # Still a bit of lag though
    time.sleep(3)
    
    # Now that we know our element exists, we can click it
    driver.find_element(By.CLASS_NAME, "yt-truncated-text__absolute-button").click()
    
    # Next we have to explicitly wait for the new content to render
    time.sleep(3)


    # Next we have to wait for the page to load
    WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.CLASS_NAME, "style-scope ytd-about-channel-renderer"))
    )

    # Still a bit of lag though
    time.sleep(3)

        
    # Now that we know our element exists, we can click it
    elements = driver.find_elements(By.XPATH, "//td[contains(@class, 'style-scope ytd-about-channel-renderer')]")
    for e_td in elements:
        # print(f"Here is data :{e_td.text}\n")
        pass
    
    elements_2 = driver.find_elements(By.XPATH, "//tr[contains(@class, 'description-item style-scope ytd-about-channel-renderer')]")
    small_dict = {}
    for e_tr in elements_2:
        yt_icon = e_tr.find_element(By.TAG_NAME, 'yt-icon')
        attribute_value = yt_icon.get_attribute("icon")
        # print(attribute_value)
        text_value = e_tr.find_elements(By.TAG_NAME, 'td')[-1]
        # print(text_value.text)

        ## Now ready to add info into small_dict
        small_dict.update({**{'ID':my_query},
                           **{attribute_value : text_value.text},
                           **dict(Scraped_DateTime=datetime.now())
                          })
        
    # Now switch from the mainpage to pdfpage
    # driver.switch_to.window(driver.window_handles[1])
    
except TimeoutException: 
    print('Redirected straight to the pdf')
    print(driver.current_url)

print(small_dict)
# finally:
#     # df = camelot_extract(pdf_url=driver.current_url)
#     df = ultimate_parser(pdf_source=driver.current_url)
#     display(df)
#     data_dict = df.to_dict(orient='index')
#     print(data_dict)
#     driver.quit()

{'ID': '@bluestar9000', 'mail': '', 'Scraped_DateTime': datetime.datetime(2025, 10, 25, 22, 43, 29, 99021), 'phone': '', 'language': 'www.youtube.com/@bluestar9000', 'privacy_public': '', 'info_outline': 'Joined 28 Jan 2007', 'person_radar': '73.4K subscribers', 'my_videos': '25 videos', 'trending_up': '51,996,777 views'}


In [174]:
from pprint import pprint
pprint(small_dict)

{'ID': '@bluestar9000',
 'Scraped_DateTime': datetime.datetime(2025, 10, 25, 22, 43, 29, 99021),
 'info_outline': 'Joined 28 Jan 2007',
 'language': 'www.youtube.com/@bluestar9000',
 'mail': '',
 'my_videos': '25 videos',
 'person_radar': '73.4K subscribers',
 'phone': '',
 'privacy_public': '',
 'trending_up': '51,996,777 views'}


# Test Iteration

In [140]:
import sys
sys.path.append('/Users/Malcolm/Documents/Helper Functions/')
import Read_n_Write as RW # We won't be using the write_list_into_txt

In [18]:
import os
import time 
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import subprocess
import pyautogui
# from tabula import read_pdf
# import camelot
import pickle # Save the scraped list of dataframes df into a pick
import time 
import chime
from datetime import datetime
now = datetime.now()

# On start 
tab_counter = 0
dfs = []
total_start = time.time()
# Start caffeinate to prevent computer from falling asleep.
caffeinate_process = subprocess.Popen('caffeinate')
print(bool(caffeinate_process))

# Convert the text data into Python list
memoized_links = RW.read_txt_into_list('Scrape_ABM') # touch this file beforehand
counter = len(memoized_links)

# Get the youtube_handles/IDs from the dataframe
youtube_handles = abm_banks['Youtube_Handle'].to_list()

True


In [19]:
total_start = time.time()
# Establish driver and service through each iteration
service = Service("/usr/local/bin/chromedriver")
driver = webdriver.Chrome(service = service)
BIG_LIST = []
with open("./Data/Scrape_ABM.txt", "a") as writer: # touch this file beforehand
    for e_youtube_handle in youtube_handles:
        my_url = f'https://www.youtube.com/{e_youtube_handle}'
        if e_youtube_handle not in memoized_links:
            start = time.time()
            if tab_counter == 0:
                # Now that we have service, let's get the first url in our list
                driver.get(my_url)
                time.sleep(2)
                # Use pyautogui to simulate the Control-Option-RightArrow
                # pyautogui.hotkey('ctrl', 'alt', 'u')    
                try:
                    # Next we have to wait for the page to load
                    WebDriverWait(driver, 15).until(
                        EC.presence_of_element_located((By.CLASS_NAME, "yt-truncated-text__absolute-button"))
                    )
                    # Still a bit of lag though
                    time.sleep(5)
                    # Now that we know our element exists, we can click it
                    driver.find_element(By.CLASS_NAME, "yt-truncated-text__absolute-button").click()
                    # Next we have to explicitly wait for the new content to render
                    time.sleep(5)
                    # Next we have to wait for the page to load
                    WebDriverWait(driver, 15).until(
                        EC.presence_of_element_located((By.CLASS_NAME, "style-scope ytd-about-channel-renderer"))
                    )
                    # Still a bit of lag though
                    time.sleep(3)

                    elements_2 = driver.find_elements(By.XPATH, "//tr[contains(@class, 'description-item style-scope ytd-about-channel-renderer')]")
                    small_dict = {}
                    for e_tr in elements_2:
                        yt_icon = e_tr.find_element(By.TAG_NAME, 'yt-icon')
                        attribute_value = yt_icon.get_attribute("icon")
                        # print(attribute_value)
                        text_value = e_tr.find_elements(By.TAG_NAME, 'td')[-1]
                        # print(text_value.text)
                
                        ## Now ready to add info into small_dict
                        small_dict.update({**{'ID':e_youtube_handle},
                                           **{attribute_value : text_value.text},
                                           **dict(Scraped_DateTime=datetime.now())
                                          })
                        chime.success()
                    BIG_LIST.append(small_dict)
                except (TimeoutException, KeyboardInterrupt): 
                    print('Timeout error occured')
                    chime.warning()
                finally:
                
                    # Write in-disk, across sessions
                    writer.write(f"{e_youtube_handle}\n")
                    writer.flush()
                    # Write in-memory, within the same session
                    memoized_links.append(e_youtube_handle)
                    counter += 1
                    print(f"The youtube channel {e_youtube_handle}, on {driver.current_url} has its table scraped.")
                    end = time.time()
                    print(f"This {counter}-th iteration took {round(end-start, 0)} seconds ")
                    
        else:
            print(f"Skiiping {e_youtube_handle}, already memoized")
            pass

## Epilogue
chime.theme('zelda')
chime.success()

# Write to dataframe 
overview_yt = pd.DataFrame(BIG_LIST)

## Write table 
with open('./Data/overview_youtube.pkl', 'wb') as f:
    pickle.dump(overview_yt, f)


The youtube channel @AFFIN, on https://www.youtube.com/@AFFIN has its table scraped.
This 1-th iteration took 17.0 seconds 
The youtube channel @AllianceBankMY, on https://www.youtube.com/@AllianceBankMY has its table scraped.
This 2-th iteration took 16.0 seconds 
The youtube channel @AmBankTV, on https://www.youtube.com/AmBankTV has its table scraped.
This 3-th iteration took 16.0 seconds 
The youtube channel @bangkokbankchannel, on https://www.youtube.com/@bangkokbankchannel has its table scraped.
This 4-th iteration took 16.0 seconds 
The youtube channel @BankofAmerica, on https://www.youtube.com/@BankofAmerica has its table scraped.
This 5-th iteration took 16.0 seconds 
The youtube channel @BankofChinaManila, on https://www.youtube.com/@BankofChinaManila has its table scraped.
This 6-th iteration took 16.0 seconds 
The youtube channel @labanquedunmondequichange, on https://www.youtube.com/@labanquedunmondequichange has its table scraped.
This 7-th iteration took 16.0 seconds 
The

TypeError: 'module' object is not callable

In [220]:
## Read back data
# Takes a while to read the pickle
import pickle
# Reading the file - unpickling
with open('./Data/overview_youtube.pkl', 'rb') as fp:
    overview_df = pickle.load(fp)

In [221]:
pd.concat([overview_df.head(), overview_df.tail()])

,ID,mail,Scraped_DateTime,phone,language,privacy_public,info_outline,person_radar,my_videos,trending_up
0,@AFFIN,,2025-10-23 19:19:33.602173,,www.youtube.com/@AFFIN,Malaysia,Joined 18 Jun 2014,4.9K subscribers,325 videos,"15,050,294 views"
1,@AllianceBankMY,,2025-10-23 19:19:49.725717,,www.youtube.com/@AllianceBankMY,Malaysia,Joined 15 Aug 2013,8.89K subscribers,398 videos,"55,406,213 views"
2,@AmBankTV,,2025-10-23 19:20:05.871238,,www.youtube.com/@AmBankTV,Malaysia,Joined 1 Jul 2013,7.35K subscribers,432 videos,"27,976,034 views"
3,@bangkokbankchannel,,2025-10-23 19:20:22.260778,,www.youtube.com/@bangkokbankchannel,Thailand,Joined 17 Apr 2012,28.8K subscribers,124 videos,"8,970,431 views"
4,@BankofAmerica,,2025-10-23 19:20:38.180865,,www.youtube.com/@BankofAmerica,United States,Joined 18 Mar 2006,122K subscribers,169 videos,"788,249 views"
19,@PublicBankGroup,,2025-10-23 19:24:59.020736,,www.youtube.com/@PublicBankGroup,Malaysia,Joined 5 Jun 2015,17.9K subscribers,217 videos,"4,743,003 views"
20,@rhbgroup,,2025-10-23 19:25:15.063088,,www.youtube.com/@rhbgroup,Malaysia,Joined 30 Jan 2010,34.8K subscribers,620 videos,"309,416,389 views"
21,@standardchartered,,2025-10-23 19:25:30.991864,,www.youtube.com/@standardchartered,United Kingdom,Joined 25 May 2009,56.9K subscribers,"1,711 videos","158,301,441 views"
22,@smbcgroup,,2025-10-23 19:25:46.879627,,www.youtube.com/@smbcgroup,Japan,Joined 3 Oct 2023,371 subscribers,22 videos,"14,889 views"
23,@uob,,2025-10-23 19:26:02.841470,,www.youtube.com/@uob,Singapore,Joined 24 Nov 2012,29.2K subscribers,"1,286 videos","86,088,593 views"


# Scraping all the video ID's
> Testing one channel

In [76]:
import os
import time 
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import subprocess
import pyautogui

In [32]:
def scrape_all_videoIDs_given_YT_handle(url, css_selector, pause_time=2, max_attempts=20):
    """
    Scrolls through a dynamically loading page and returns all matching elements.

    Use case : Get all the video_IDS of a given channel via the infinite scroll method.
    
    
    Args:
        url (str): Target webpage URL like "https://www.youtube.com/@AmBankTV/videos"
        css_selector (str): CSS selector for elements you want to extract
        pause_time (float): Time (seconds) to wait between scrolls
        max_attempts (int): Max consecutive scrolls without new content before stopping

    Returns:
        list of WebElement: All elements matching the selector
    """
    # Establish driver and service through each iteration
    service = Service("/usr/local/bin/chromedriver")
    driver = webdriver.Chrome(service = service)
    driver.get(url)
    # Use pyautogui to simulate the Control-Option-RightArrow
    pyautogui.hotkey('ctrl', 'alt', 'enter')   
    wait = WebDriverWait(driver, 10)
    
    # Wait until at least one element appears
    wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "yt-truncated-text__absolute-button")))
    # Still a bit of lag though
    time.sleep(3)
    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    same_height_attempts = 0
    
    print("🔄 Starting infinite scroll...")

    while same_height_attempts < max_attempts:
        # Scroll to the bottom
        # driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        
        # Wait for page to load new items
        time.sleep(pause_time)
        
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        
        # Wait for page to load new items
        time.sleep(pause_time)
        
        if new_height == last_height:
            same_height_attempts += 1
            print(f"⚠️ No new content... ({same_height_attempts}/{max_attempts})")
        else:
            same_height_attempts = 0
            last_height = new_height
            print("✅ New content loaded.")
    
    print("🛑 Finished scrolling.")
    
    # Collect all loaded elements
    elements = driver.find_elements(By.XPATH, css_selector)
    
    print(f"📦 Total elements found: {len(elements)}")
    
    return elements, driver  # return driver if you want to extract text/attributes later


## Invocation
my_youtube_handle = f"@AmBankTV"
my_selector="//ytd-rich-item-renderer[contains(@class, 'style-scope ytd-rich-grid-renderer')]"
my_elements, my_driver = scrape_all_videoIDs_given_YT_handle(url=fr"https://www.youtube.com/{my_youtube_handle}/videos",
                                                             css_selector=my_selector,
                                                             pause_time=5,
                                                             max_attempts=3)

# Appending to a list 
video_links = []
for el in my_elements:
    yt_icon = el.find_element(By.TAG_NAME, 'a')
    attribute_value = yt_icon.get_attribute("href")
    print(attribute_value)
    video_links.append(attribute_value)

## Write the list into a txt file
timestamp = datetime.now().strftime("%m/%d/%Y, %H:%M:%S)
RW.write_list_into_txt(senarai=video_links, filename=f"{my_youtube_handle}_videoIDs_{timestamp}") 


🔄 Starting infinite scroll...
✅ New content loaded.
✅ New content loaded.
✅ New content loaded.
✅ New content loaded.
✅ New content loaded.
✅ New content loaded.
✅ New content loaded.
✅ New content loaded.
✅ New content loaded.
✅ New content loaded.
✅ New content loaded.
✅ New content loaded.
⚠️ No new content... (1/3)
⚠️ No new content... (2/3)
⚠️ No new content... (3/3)
🛑 Finished scrolling.
📦 Total elements found: 362
https://www.youtube.com/watch?v=p7bPO1SfHJc&pp=0gcJCQYKAYcqIYzv
https://www.youtube.com/watch?v=xceiS_r8WlQ
https://www.youtube.com/watch?v=s9pNJYzRvxs
https://www.youtube.com/watch?v=FPGhDYBIlVw
https://www.youtube.com/watch?v=obl5r-aOqy4
https://www.youtube.com/watch?v=sh0XVQDBBsY
https://www.youtube.com/watch?v=-EmdeyJqKs0
https://www.youtube.com/watch?v=lyUjdro80JU
https://www.youtube.com/watch?v=Og6vcBivg9s
https://www.youtube.com/watch?v=iWlD8iSavE8&pp=0gcJCQYKAYcqIYzv
https://www.youtube.com/watch?v=Gxqo6ATyNRk
https://www.youtube.com/watch?v=dakhg2Y5dmA
https:/

# Big Scrape 1
> Scraping all video IDs

In [150]:
## Scrape all video_IDs
youtube_handles = overview_df['ID'].to_list()

## Invocation
my_selector="//ytd-rich-item-renderer[contains(@class, 'style-scope ytd-rich-grid-renderer')]"

for e_yt_handle in youtube_handles:
    my_elements, my_driver = scrape_all_videoIDs_given_YT_handle(url=fr"https://www.youtube.com/{e_yt_handle}/videos",
                                                                 css_selector=my_selector,
                                                                 pause_time=5,
                                                                 max_attempts=3)
    # Appending to a list 
    video_links = []
    for el in my_elements:
        yt_icon = el.find_element(By.TAG_NAME, 'a')
        attribute_value = yt_icon.get_attribute("href")
        print(attribute_value)
        video_links.append(attribute_value)
    
    ## Write the list into a txt file
    RW.write_list_into_txt(senarai=video_links,
                           filename=f"./Video_IDs_per_channel/{e_yt_handle}_{datetime.now().strftime('%m-%d-%Y_%H:%M:%S')}") # touch this file

🔄 Starting infinite scroll...
✅ New content loaded.
✅ New content loaded.
✅ New content loaded.
✅ New content loaded.
✅ New content loaded.
✅ New content loaded.
✅ New content loaded.
⚠️ No new content... (1/3)
⚠️ No new content... (2/3)
⚠️ No new content... (3/3)
🛑 Finished scrolling.
📦 Total elements found: 221
https://www.youtube.com/watch?v=RMZdBov0FrQ
https://www.youtube.com/watch?v=Hhi_ERnrv1g
https://www.youtube.com/watch?v=t8UCJJafTd0
https://www.youtube.com/watch?v=PyuTTP4C12c
https://www.youtube.com/watch?v=SPZ4q1CnxnE
https://www.youtube.com/watch?v=ENrC21on7Gw
https://www.youtube.com/watch?v=PXGs538p-rQ
https://www.youtube.com/watch?v=5Is2nERAA34
https://www.youtube.com/watch?v=fI6u0JPz8ME&pp=0gcJCQYKAYcqIYzv
https://www.youtube.com/watch?v=wNDPH4Fawao
https://www.youtube.com/watch?v=56rQD9GJKA4
https://www.youtube.com/watch?v=izaCO4ob_ao
https://www.youtube.com/watch?v=zjmW3Uqgt8I
https://www.youtube.com/watch?v=PYdQF1odVKY
https://www.youtube.com/watch?v=q2O12vu774U
https

# Validate videos
> Check that video count in txt files matches `overview['my_vidoes']`
> > It doesn't due to Youtube Shorts

In [208]:
import os
import re
directory = r"./Data/Video_IDs_per_channel/"
videos_not_shorts = {}
for e_file in os.listdir(directory):
    match = re.search(pattern=r"^((@[\w]+)_([\d\-]+)_([\d\:]+))\.txt$", string=e_file)
    if match:
        print(f"Match groups : {match.groups()}")
        e_list = RW.read_txt_into_list(filename=fr"./Video_IDs_per_channel/{match.group(1)}")
        videos_not_shorts[match.group(2)] = len(e_list)
    else:
        pass

Match groups : ('@standardchartered_10-24-2025_23:57:16', '@standardchartered', '10-24-2025', '23:57:16')
Match groups : ('@rhbgroup_10-24-2025_23:47:16', '@rhbgroup', '10-24-2025', '23:47:16')
Match groups : ('@GXBank_10-24-2025_23:15:04', '@GXBank', '10-24-2025', '23:15:04')
Match groups : ('@jpmorgan_10-24-2025_23:26:04', '@jpmorgan', '10-24-2025', '23:26:04')
Match groups : ('@AFFIN_10-24-2025_22:24:16', '@AFFIN', '10-24-2025', '22:24:16')
Match groups : ('@uob_10-25-2025_00:05:48', '@uob', '10-25-2025', '00:05:48')
Match groups : ('@MyBoostApp_10-24-2025_22:49:47', '@MyBoostApp', '10-24-2025', '22:49:47')
Match groups : ('@HSBC_MY_10-24-2025_23:18:18', '@HSBC_MY', '10-24-2025', '23:18:18')
Match groups : ('@smbcgroup_10-24-2025_23:57:52', '@smbcgroup', '10-24-2025', '23:57:52')
Match groups : ('@hongleongbankmy_10-24-2025_23:16:41', '@hongleongbankmy', '10-24-2025', '23:16:41')
Match groups : ('@BankofAmerica_10-24-2025_22:31:34', '@BankofAmerica', '10-24-2025', '22:31:34')
Match 

In [214]:
my_loc = overview_df.columns.get_loc('my_videos')
# Insert a new column 'new_col' after 'colB'
# The location will be loc_colB + 1
overview_df.insert(loc=my_loc+1, column='Videos_Not_Shorts', value=overview_df['ID'].apply(lambda x: videos_not_shorts.get(x, 'Not Found')))

In [217]:
## Write table 
with open('./Data/overview_youtube_2.pkl', 'wb') as f:
    pickle.dump(overview_df, f)

In [7]:
## Read back data
# Takes a while to read the pickle
import pickle
# Reading the file - unpickling
with open('./Data/overview_youtube_2.pkl', 'rb') as fp:
    overview_df = pickle.load(fp) # New and improved 

In [9]:
overview_df.head(2)

,ID,mail,Scraped_DateTime,phone,language,privacy_public,info_outline,person_radar,my_videos,Videos_Not_Shorts,trending_up
0,@AFFIN,,2025-10-23 19:19:33.602173,,www.youtube.com/@AFFIN,Malaysia,Joined 18 Jun 2014,4.9K subscribers,325 videos,221,"15,050,294 views"
1,@AllianceBankMY,,2025-10-23 19:19:49.725717,,www.youtube.com/@AllianceBankMY,Malaysia,Joined 15 Aug 2013,8.89K subscribers,398 videos,356,"55,406,213 views"


# Data cleaning overview_df

## Creating databases

In [339]:
import os
import re
directory = r"./Data/Video_IDs_per_channel/"
videos_not_shorts = {}
for e_file in os.listdir(directory):
    match = re.search(pattern=r"^((@[\w]+)_([\d\-]+)_([\d\:]+))\.txt$", string=e_file)
    if match:
        print(f"Match groups : {match.groups()}")
        e_list = RW.read_txt_into_list(filename=fr"./Video_IDs_per_channel/{match.group(1)}")
        videos_not_shorts[match.group(2)] = len(e_list)
        # Create empty dataframe
        df = pd.DataFrame()

        IDs = []
        for e_url in e_list:
            e_ID = re.search(pattern=r"\?v=(.*)$", string=e_url).group(1)
            IDs.append(e_ID)
        
        df['Youtube_Video_IDs'] = IDs
        df = df.assign(Youtube_Handle=str(match.group(2)))
        ## Write table 
        with open(f'./Data/Database/{match.group(2)}.pkl', 'wb') as f:
            pickle.dump(df, f)

Match groups : ('@standardchartered_10-24-2025_23:57:16', '@standardchartered', '10-24-2025', '23:57:16')
Match groups : ('@rhbgroup_10-24-2025_23:47:16', '@rhbgroup', '10-24-2025', '23:47:16')
Match groups : ('@GXBank_10-24-2025_23:15:04', '@GXBank', '10-24-2025', '23:15:04')
Match groups : ('@jpmorgan_10-24-2025_23:26:04', '@jpmorgan', '10-24-2025', '23:26:04')
Match groups : ('@AFFIN_10-24-2025_22:24:16', '@AFFIN', '10-24-2025', '22:24:16')
Match groups : ('@uob_10-25-2025_00:05:48', '@uob', '10-25-2025', '00:05:48')
Match groups : ('@MyBoostApp_10-24-2025_22:49:47', '@MyBoostApp', '10-24-2025', '22:49:47')
Match groups : ('@HSBC_MY_10-24-2025_23:18:18', '@HSBC_MY', '10-24-2025', '23:18:18')
Match groups : ('@smbcgroup_10-24-2025_23:57:52', '@smbcgroup', '10-24-2025', '23:57:52')
Match groups : ('@hongleongbankmy_10-24-2025_23:16:41', '@hongleongbankmy', '10-24-2025', '23:16:41')
Match groups : ('@BankofAmerica_10-24-2025_22:31:34', '@BankofAmerica', '10-24-2025', '22:31:34')
Match 

In [393]:
overview_df
for idx, row_dict in overview_df.to_dict(orient='index').items():
    print(row_dict['ID'])

@AFFIN
@AllianceBankMY
@AmBankTV
@bangkokbankchannel
@BankofAmerica
@BankofChinaManila
@labanquedunmondequichange
@MyBoostApp
@CIMBGroupHoldingsBhd
@Citi
@DeutscheBank
@GXBank
@hongleongbankmy
@HSBC_MY
@jpmorgan
@maybankvideos
@MizuhoAmericas
@MUFGBankChannel
@channelocbc
@PublicBankGroup
@rhbgroup
@standardchartered
@smbcgroup
@uob


In [396]:
pd.concat([overview_df.head(), overview_df.tail()], axis='columns')

,ID,mail,Scraped_DateTime,phone,language,privacy_public,info_outline,person_radar,my_videos,trending_up,ID,mail,Scraped_DateTime,phone,language,privacy_public,info_outline,person_radar,my_videos,trending_up
0,@AFFIN,,2025-10-23 19:19:33.602173,,www.youtube.com/@AFFIN,Malaysia,Joined 18 Jun 2014,4.9K subscribers,325 videos,"15,050,294 views",NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,@AllianceBankMY,,2025-10-23 19:19:49.725717,,www.youtube.com/@AllianceBankMY,Malaysia,Joined 15 Aug 2013,8.89K subscribers,398 videos,"55,406,213 views",NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,@AmBankTV,,2025-10-23 19:20:05.871238,,www.youtube.com/@AmBankTV,Malaysia,Joined 1 Jul 2013,7.35K subscribers,432 videos,"27,976,034 views",NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,@bangkokbankchannel,,2025-10-23 19:20:22.260778,,www.youtube.com/@bangkokbankchannel,Thailand,Joined 17 Apr 2012,28.8K subscribers,124 videos,"8,970,431 views",NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,@BankofAmerica,,2025-10-23 19:20:38.180865,,www.youtube.com/@BankofAmerica,United States,Joined 18 Mar 2006,122K subscribers,169 videos,"788,249 views",NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,@PublicBankGroup,,2025-10-23 19:24:59.020736,,www.youtube.com/@PublicBankGroup,Malaysia,Joined 5 Jun 2015,17.9K subscribers,217 videos,"4,743,003 views"
20,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,@rhbgroup,,2025-10-23 19:25:15.063088,,www.youtube.com/@rhbgroup,Malaysia,Joined 30 Jan 2010,34.8K subscribers,620 videos,"309,416,389 views"
21,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,@standardchartered,,2025-10-23 19:25:30.991864,,www.youtube.com/@standardchartered,United Kingdom,Joined 25 May 2009,56.9K subscribers,"1,711 videos","158,301,441 views"
22,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,@smbcgroup,,2025-10-23 19:25:46.879627,,www.youtube.com/@smbcgroup,Japan,Joined 3 Oct 2023,371 subscribers,22 videos,"14,889 views"
23,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,@uob,,2025-10-23 19:26:02.841470,,www.youtube.com/@uob,Singapore,Joined 24 Nov 2012,29.2K subscribers,"1,286 videos","86,088,593 views"


## Scraping a video for likes comments views 

In [2]:
import re
def correct_number(number_string):
    if re.search(pattern=r"^\d{1,3}(?:,\d{3})*(?:\.\d*)?$", string=number_string):
        temp = number_string.replace(',', '')
        good_float = int(temp)
    return good_float

correct_number('1,000')

1000

## Working 
> Only works for primary OP poster

In [3]:
## Disabled comments
url = "https://www.youtube.com/watch?v=3RKCWeOpip0"

## Normal
# url = r"https://www.youtube.com/watch?v=sIYNr_B1TEQ"

## Descriptionless video
# url = r"https://www.youtube.com/watch?v=xMBJOEU29Zw"

## Zero Comments
# url = r"https://www.youtube.com/watch?v=QjXMxlR8zKo"



service = Service("/usr/local/bin/chromedriver")
driver = webdriver.Chrome(service=service)
driver.get(url)

# (Optional) move browser window using pyautogui
pyautogui.hotkey('ctrl', 'alt', 'left')

wait = WebDriverWait(driver, 30)

more_info = (By.XPATH, '//tp-yt-paper-button[contains(@id, "expand")]')
class1 = (By.CLASS_NAME, "button style-scope ytd-text-inline-expander")
xpath_2 = (By.XPATH, "//yt-formatted-string[contains(@id, 'info')]")
comments_path = (By.XPATH, "//ytd-comment-view-model[contains(@id, 'comment')]")
reply_comments_path = (By.XPATH, ".//div[contains(@id, 'collapsed-threads')]")
like_button_path = (By.XPATH, "//button[contains(@aria-label, 'like')]")
replies_button = (By.XPATH, "//button[contains(@aria-label, 'replies')]")


MM_collector = {}

# Next we have to wait for the page to load
WebDriverWait(driver, 15).until(
    EC.presence_of_element_located(more_info)
)
# Still a bit of lag though
time.sleep(3)

## ---------------VIEWS_AND_DATE----------------------------
# Now that we know our element exists, we can click it -THIS DOES NOT WORK
# Check which elements are actually interactable
elements = driver.find_elements(*more_info) # use plural
print("---------------VIEWS_AND_DATE----------------------------")
for i, el in enumerate(elements):
    rect = el.rect
    displayed = el.is_displayed()
    print(f"[{i}] Tag: {el.tag_name}, Visible: {displayed}, Rect: {rect}")
    if displayed:
        clickable_element = el

# From above we know the last element is the interactable element we can click to expand : driver.find_elements(*more_info)[-1].click()
# Check if we found any interactable element
if clickable_element:
    # Scroll into view before clicking (good practice)
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", clickable_element)
    clickable_element.click()
    print("✅ Clicked visible 'expand' element.")

# Next we have to explicitly wait for the new content to render
time.sleep(5)
# Next we have to wait for the page to load
wait.until(
    EC.presence_of_element_located(more_info)
)

# Collect all loaded elements
views_and_date = driver.find_elements(*xpath_2)[-1].find_elements(By.XPATH, ".//span[contains(@class, 'style-scope yt-formatted-string')]")
print("---------------VIEWS_AND_DATE 2---------------------------")
if views_and_date:
    for idx, e_child_ele in enumerate(views_and_date, start=1):
        some_info = e_child_ele.text
        print(some_info)
        if idx == 1 and some_info:
            MM_collector.update({'views' : some_info})
        elif idx == 3 and some_info:
            MM_collector.update({'date' : some_info})
else: # For descriptionless videos
    try:
        metadata_block = wait.until(EC.presence_of_element_located(
            (By.XPATH, "//ytd-watch-metadata")
        ))
    
        # Search for views and date strings inside that block
        info_spans = metadata_block.find_elements(
            By.XPATH, ".//span[contains(@class, 'style-scope yt-formatted-string')]"
        )
    
        print("---------------VIEWS_AND_DATE----------------------------")
        for idx, span in enumerate(info_spans, start=1):
            text = span.text.strip()
            if "views" in text:
                MM_collector["views"] = text
            elif any(k in text.lower() for k in ["ago", "202", "201", "2025"]):
                MM_collector["date"] = text
            print(f"{idx:02d}: {text}")
    except Exception as e:
        print("❌ Could not extract metadata:", e)


## --------------- TITLE ---------------------------------
title = driver.find_elements(By.XPATH, "//yt-formatted-string[@title]")
print("--------------- TITLE ---------------------------------")
for i, el in enumerate(title):
    rect = el.rect
    displayed = el.is_displayed()
    print(f"[{i}] Tag: {el.tag_name}, Visible: {displayed}, Rect: {rect}")
    if displayed:
        # Get the first visible element
        clickable_element = el
        break
MM_collector.update({'title' :  clickable_element.get_attribute('title')})

## --------------- LIKE-BUTTON ----------------------------
# First grab all buttons with aria-label attribute
like_button = driver.find_elements(*like_button_path)
print("--------------- LIKE-BUTTON ---------------------------------")
for i, el in enumerate(like_button):
    rect = el.rect
    displayed = el.is_displayed()
    print(f"[{i}] Tag: {el.tag_name}, Visible: {displayed}, Rect: {rect}")
    if displayed:
        clickable_element = el
        break

# Then filter with regex
pattern = re.compile(r"^like.*", re.IGNORECASE)
like_info_match = pattern.search(string=clickable_element.get_attribute("aria-label"))
if like_info_match:
    like_string = clickable_element.get_attribute('aria-label')
    print(like_string)
    thousand_comma_separation_pattern = re.compile(r"\d{1,3}(?:,\d{3})*(?:\.\d*)?", re.IGNORECASE)
    number = thousand_comma_separation_pattern.search(string=like_string)
    print(f"Here is the like-count : {correct_number(number.group(0))}")
    MM_collector.update({'likes' : correct_number(number.group(0))})

##---------------DESCRIPTION ----------------------------
# Inspect which elements are interactable
description = driver.find_elements(By.XPATH, "//div[contains(@id, 'expanded')]") # First path
for i, el in enumerate(description):
    rect = el.rect
    displayed = el.is_displayed()
    print(f"[{i}] Tag: {el.tag_name}, Visible: {displayed}, Rect: {rect}")
    if displayed:
        clickable_element = el

try:
    # Use relative ".//" to search within nested elements/descendants 'under' descripton[-1]
    many_spans = clickable_element.find_element(By.XPATH, ".//yt-attributed-string[contains(@class, 'ytd-text-inline-expander')]")\
                                  .find_elements(By.XPATH, ".//span[contains(@class, 'yt-core-attributed-string--link-inherit-color')]") # third path
    
    description_list = []
    count = 0
    for e_span in many_spans:
        clean_str = e_span.text.strip()
        description_list.append(clean_str)
        description_string = '\n'.join(description_list)
    
        count += 1
    
    print(f"This is the description:\n{description_string}")
except NoSuchElementException:
    description_string = 'No description has been added to this video'
    
MM_collector.update({'description' : description_string, 
                     'MM_timestamp' : datetime.now().strftime("%m/%d/%Y %H:%M:%S")
                    })

max_num_of_scrolls = None
num_of_scrolls = 0
pause_time=2
max_attempts=2

last_height = driver.execute_script("return document.documentElement.scrollHeight")
same_height_attempts = 0
print("🔄 Starting infinite scroll...")

while same_height_attempts < max_attempts:
    if num_of_scrolls == max_num_of_scrolls:
        print(f"Max num of scrolls reached. Breaking out.")
        break
    # Scroll to the bottom
    # driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    
    # Wait for page to load new items
    time.sleep(pause_time)
    
    # (Optional) move browser window using pyautogui
    pyautogui.hotkey('ctrl', 'alt', 'left')
    
    # Wait for page to load new items
    time.sleep(pause_time)
    
    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    
    # Wait for page to load new items
    time.sleep(pause_time)
    
    if new_height == last_height:
        same_height_attempts += 1
        print(f"⚠️ No new content... ({same_height_attempts}/{max_attempts})")
    else:
        same_height_attempts = 0
        last_height = new_height
        print("✅ New content loaded.")
        num_of_scrolls += 1


# Collect all loaded comments
COMMENT_DICT = []
comments = driver.find_elements(*comments_path)

for el in comments:
    emojies_string = []
    # Get any emojis
    emojies = el.find_elements(By.XPATH, ".//img[@alt]")
    for e_emoji in emojies:
        print(e_emoji.get_attribute('alt'))
        actual_emoji = e_emoji.get_attribute('alt')
        emojies_string.append(actual_emoji)
    # Get the replies 
    replies = el.find_elements(*reply_comments_path)
    # print(replies.text)
    small_comment_dict = {}
    # Process the text
    li = el.text.split('\n')
    lit = list(filter(lambda x : not re.search(pattern=r"^Pinned", string=x), li))

    # Deal with unliked comments
    if not re.search(pattern=r'^\d+$', string=lit[-2]):
        print("Unliked comment")
        lit.insert(-1, '0')
    else:
        pass
    print(lit)
        
        
    poster, comment_age, *comment_text_content, like_count, reply = lit
    full_comment = ' '.join(comment_text_content)
    emoji_comment = ' '.join(emojies_string)
    full_comment += emoji_comment
    small_comment_dict.update({'poster_OP' : poster,
                               'comment_age' : comment_age,
                               'full_comment' : full_comment,
                               'like_count' : like_count
                              })
    COMMENT_DICT.append(small_comment_dict)
    print("-----")

# Done can exit window
driver.quit()

# pprint(MM_collector)
# pprint(COMMENT_DICT)

NameError: name 'Service' is not defined

In [4]:
pd.DataFrame(data=COMMENT_DICT)

NameError: name 'pd' is not defined

In [5]:
pd.DataFrame.from_dict(data=pd.Series(MM_collector)).T

NameError: name 'pd' is not defined

In [674]:


import pandas as pd
import numpy as np

data = {'A': [1, 2, 3], 'B': [10, 20, 30]}
df = pd.DataFrame(data)
display(df)

# Apply a function (e.g., sum) along columns
column_sums = df.apply(np.sum, axis=0)
print("Column sums:\n")
display(column_sums)

,A,B
0,1,10
1,2,20
2,3,30


Column sums:



A     6
B    60
dtype: int64

In [867]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

driver = webdriver.Chrome()
driver.get("https://www.youtube.com/watch?v=dQw4w9WgXcQ")

# Wait briefly for the page to load
time.sleep(3)

try:
    # Works for watch pages too, sometimes visible in overlay
    duration_element = driver.find_element(
        By.CSS_SELECTOR, 'span.ytp-time-duration'
    )
    print("Video length:", duration_element.text)
except Exception as e:
    print("Could not find duration:", e)

driver.quit()


Video length: 3:33


## Function to create databases

In [1]:
import re
from dateutil.parser import parse
International_date = r"\d{1,2},?\s+[A-Za-z]{3,9},?\s+\d{4}"
US_date = r"[A-Za-z]{3,9},?\s+\d{1,2},?\s+\d{4}"

tc = ['12 Sept, 2023', 'Mar 14 2025', '14 September 2023', 'Sep 1, 2025', 'Premiered on Jan 13 2025', 'Premiered Mar 19, 2024']
for e_tc in tc:
    match = re.search(pattern=rf'\b(?:{International_date}|{US_date})\b$', string=e_tc) #[A-Z-a-z\s]*[^\w]*
    print(match.group(), parse(match.group()).date())
    

print(rf'\b(?:{International_date}|{US_date})\b')

12 Sept, 2023 2023-09-12
Mar 14 2025 2025-03-14
14 September 2023 2023-09-14
Sep 1, 2025 2025-09-01
Jan 13 2025 2025-01-13
Mar 19, 2024 2024-03-19
\b(?:\d{1,2},?\s+[A-Za-z]{3,9},?\s+\d{4}|[A-Za-z]{3,9},?\s+\d{1,2},?\s+\d{4})\b


In [2]:
import re
def correct_number(number_string):
    if re.search(pattern=r"^\d{1,3}(?:,\d{3})*(?:\.\d*)?$", string=number_string):
        temp = number_string.replace(',', '')
        good_float = int(temp)
    return good_float

correct_number('1,000')

1000

In [3]:
import sys
sys.path.append('/Users/Malcolm/Documents/Helper Functions/')
import Read_n_Write as RW # We won't be using the write_list_into_txt

In [4]:
import os
import time 
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import subprocess
import pyautogui
# from tabula import read_pdf
# import camelot
import pickle # Save the scraped list of dataframes df into a pick
import time 
from datetime import datetime
import pandas as pd
now = datetime.now()


In [5]:
## Create function 
# Call 
from dateutil.parser import parse
from pytube import YouTube

def scrape_into_2_databases(url_ID, is_print=False):
    """
    """

    url = fr"https://www.youtube.com/watch?v={url_ID}"
    yt = YouTube(url)

    service = Service("/usr/local/bin/chromedriver")
    driver = webdriver.Chrome(service=service)
    driver.get(url)
    
    # (Optional) move browser window using pyautogui
    pyautogui.hotkey('ctrl', 'alt', 'left')
    
    wait = WebDriverWait(driver, 30)
    
    more_info = (By.XPATH, '//tp-yt-paper-button[contains(@id, "expand")]')
    class1 = (By.CLASS_NAME, "button style-scope ytd-text-inline-expander")
    xpath_2 = (By.XPATH, "//yt-formatted-string[contains(@id, 'info')]")
    comments_path = (By.XPATH, "//ytd-comment-view-model[contains(@id, 'comment')]")
    reply_comments_path = (By.XPATH, ".//div[contains(@id, 'collapsed-threads')]")
    like_button_path = (By.XPATH, "//button[contains(@aria-label, 'like')]")
    replies_button = (By.XPATH, "//button[contains(@aria-label, 'replies')]")
    
    
    MM_collector = {}
    COMMENT_DICT = []

    
    # Next we have to wait for the page to load
    try: # Firstly check if the video is NOT Private

        # Next we have to wait for the page to load
        wait.until(
            EC.presence_of_element_located(xpath_2)
        )

    except TimeoutException: # Handle if it is Private
        try:
            wait.until(
                EC.presence_of_element_located((By.XPATH, "//yt-button-renderer[contains(@id, 'button')]"))
            )
             # Still a bit of lag though
            time.sleep(3)
            sign_in_btn = driver.find_elements(By.XPATH, "//yt-button-renderer[contains(@id, 'button')]") 
            for i, el in enumerate(sign_in_btn):
                rect = el.rect
                displayed = el.is_displayed()
                if is_print:
                    print(f"[{i}] Tag: {el.tag_name}, Visible: {displayed}, Rect: {rect}")
                if displayed:
                    clickable_element = el
                    
            if clickable_element:
                print('A private video!!')
                MM_collector.update({'video_length' : None,
                                     'date': pd.NaT,
                                     'views':pd.NA,	
                                     'title':None,	
                                     'likes':pd.NA,	
                                     'description':None,
                                     'MM_timestamp':pd.NaT,
                                     'is_commented':None
                                    })
            return pd.DataFrame.from_dict(data=pd.Series(MM_collector)).T, pd.DataFrame(data=COMMENT_DICT)
        except TimeoutException:
            driver.quit()
            time.sleep(60)
            scrape_into_2_databases(url_ID, is_print=False)
            
        
        
    # Still a bit of lag though
    time.sleep(3)

    ## ---------------VIDEO LENGTH----------------------------
    print('## ---------------VIDEO LENGTH----------------------------')
    try:
        # Works for watch pages too, sometimes visible in overlay
        duration_element = driver.find_element(
            By.CSS_SELECTOR, 'span.ytp-time-duration'
        )
        print("Video length:", duration_element.text)
        if duration_element.text:
            MM_collector.update({'video_length' : duration_element.text })
        else:
             MM_collector.update({'video_length' : yt.length})
    except Exception as e:
        print("Could not find duration:", e)
    
    ## ---------------VIEWS_AND_DATE----------------------------
    # Now that we know our element exists, we can click it -THIS DOES NOT WORK
    # Check which elements are actually interactable
    elements = driver.find_elements(*more_info) # use plural
    print("---------------VIEWS_AND_DATE----------------------------")
    
    for i, el in enumerate(elements):
        rect = el.rect
        displayed = el.is_displayed()
        if is_print:
            print(f"[{i}] Tag: {el.tag_name}, Visible: {displayed}, Rect: {rect}")
        if displayed:
            clickable_element = el
    
    # From above we know the last element is the interactable element we can click to expand : driver.find_elements(*more_info)[-1].click()
    # Check if we found any interactable element
    if clickable_element:
        # Scroll into view before clicking (good practice)
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", clickable_element)
        clickable_element.click()
        print("✅ Clicked visible 'expand' element.")
    
    # Next we have to explicitly wait for the new content to render
    time.sleep(5)
    # Next we have to wait for the page to load
    wait.until(
        EC.presence_of_element_located(more_info) 
    )
    # Next we have to explicitly wait for the new content to render
    time.sleep(3)
    
    # Collect all loaded elements
    views_and_date = driver.find_elements(*xpath_2)[-1].find_elements(By.XPATH, ".//span[contains(@class, 'style-scope yt-formatted-string')]")
    print("---------------VIEWS_AND_DATE 2---------------------------")
    International_date = r"\d{1,2},?\s+[A-Za-z]{3,9},?\s+\d{4}"
    US_date = r"[A-Za-z]{3,9},?\s+\d{1,2},?\s+\d{4}"
    if views_and_date:
        
        for idx, e_child_ele in enumerate(views_and_date, start=1):
            some_info = e_child_ele.text
            views_match = re.search(pattern=r"^\d{1,3}(?:,\d{3})*\s+(?:\w*)?$", string=some_info)
            date_match = re.search(pattern=rf'\b(?:{International_date}|{US_date})\b$', string=some_info) #[A-Z-a-z\s]*[^\w]*
            print(some_info)
            if views_match: # idx == 1 and some_info
                MM_collector.update({'views' : views_match.group()})
            elif date_match:
                MM_collector.update({'date' : date_match.group()})
    else: # For descriptionless videos
        try:
            metadata_block = wait.until(EC.presence_of_element_located(
                (By.XPATH, "//ytd-watch-metadata")
            ))
        
            # Search for views and date strings inside that block
            info_spans = metadata_block.find_elements(
                By.XPATH, ".//span[contains(@class, 'style-scope yt-formatted-string')]"
            )
        
            print("---------------VIEWS_AND_DATE for basic videos----------------------------")
            for idx, span in enumerate(info_spans, start=1):
                some_info = span.text.strip()
                views_match = re.search(pattern=r"^\d{1,3}(?:,\d{3})*\s+(?:\w*)?$", string=some_info)
                date_match = re.search(pattern=rf'\b(?:{International_date}|{US_date})\b$', string=some_info) #[A-Z-a-z\s]*[^\w]*
                
                if views_match: #"views" in text
                    MM_collector.update({'views' : views_match.group()})
                    print(f"{idx:02d}: {views_match.group(0)}")
                elif date_match: 
                    MM_collector.update({'date' : date_match.group()})
                    print(f"{idx:02d}: {date_match.group(0)}")
                elif any(k in some_info.lower() for k in ["ago", "202", "201", "2025"]):
                    MM_collector["date"] = some_info
                    pass
                    print(f"{idx:02d}: {some_info}")
                  
        except Exception as e:
            print("❌ Could not extract metadata:", e)

    ## --------------- TITLE ---------------------------------
    title = driver.find_elements(By.XPATH, "//yt-formatted-string[@title]")
    for i, el in enumerate(title):
        rect = el.rect
        displayed = el.is_displayed()
        if is_print:
            print(f"[{i}] Tag: {el.tag_name}, Visible: {displayed}, Rect: {rect}")
        if displayed:
            # Get the first visible element
            clickable_element = el
            break
    MM_collector.update({'title' :  clickable_element.get_attribute('title')})
    ## ---------------LIKE-BUTTON ----------------------------
    # First grab all buttons with aria-label attribute
    like_button = driver.find_elements(*like_button_path)
    for i, el in enumerate(like_button):
        rect = el.rect
        displayed = el.is_displayed()
        if is_print:
            print(f"[{i}] Tag: {el.tag_name}, Visible: {displayed}, Rect: {rect}")
        if displayed:
            clickable_element = el
            break
    
    # Then filter with regex
    pattern = re.compile(r"^like.*", re.IGNORECASE)
    like_info_match = pattern.search(string=clickable_element.get_attribute("aria-label"))
    if like_info_match:
        like_string = clickable_element.get_attribute('aria-label')
        thousand_comma_separation_pattern = re.compile(r"\d{1,3}(?:,\d{3})*(?:\.\d*)?", re.IGNORECASE)
        number = thousand_comma_separation_pattern.search(string=like_string)
        print(f"Here is the like-count : {correct_number(number.group(0))}")
        MM_collector.update({'likes' : correct_number(number.group(0))})
    else: # Need to deal with the case that the video has no likes
        MM_collector.update({'likes' : 0})
    
    ##---------------DESCRIPTION ----------------------------
    # Inspect which elements are interactable
    description = driver.find_elements(By.XPATH, "//div[contains(@id, 'expanded')]") # First path
    for i, el in enumerate(description):
        rect = el.rect
        displayed = el.is_displayed()
        if is_print:
            print(f"[{i}] Tag: {el.tag_name}, Visible: {displayed}, Rect: {rect}")
        if displayed:
            clickable_element = el

    try:
        # Use relative ".//" to search within nested elements/descendants 'under' descripton[-1]
        many_spans = clickable_element.find_element(By.XPATH, ".//yt-attributed-string[contains(@class, 'ytd-text-inline-expander')]")\
                                      .find_elements(By.XPATH, ".//span[contains(@class, 'yt-core-attributed-string--link-inherit-color')]") # third path
        
        description_list = []
        count = 0
        for e_span in many_spans:
            clean_str = e_span.text.strip()
            description_list.append(clean_str)
            description_string = '\n'.join(description_list)
            count += 1
    except NoSuchElementException:
        description_string = 'No description has been added to this video'
    
    print(f"This is the description:\n{description_string}")
    MM_collector.update({'description' : description_string,
                         'MM_timestamp' : datetime.now().strftime("%m/%d/%Y %H:%M:%S")
                        })
    # ---------- SCROLLING -------------------------
    max_num_of_scrolls = None
    num_of_scrolls = 0 # Track the num of scrolls AND HEIGHT REFRESHES/CHANGE
    pause_time = 2
    max_attempts = 1
    
    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    same_height_attempts = 0
    
    # Track how many comment elements are present
    last_comment_count = 0
    
    print("🔄 Starting infinite scroll...")
    
    is_scrape_comments = True
    
    while same_height_attempts < max_attempts:
        if max_num_of_scrolls and num_of_scrolls >= max_num_of_scrolls:
            print("🛑 Max num of scrolls reached. Breaking out.")
            break
    
        # Scroll to bottom
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(pause_time)
    
        # Optional: move window back into view (if YouTube switches focus)
        try:
            pyautogui.hotkey('ctrl', 'alt', 'left')
            time.sleep(0.5)
        except Exception:
            pass
    
        # Check how many comments are currently loaded
        comment_elements = driver.find_elements(By.XPATH, "//ytd-comment-view-model[contains(@id, 'comment')]")
        current_comment_count = len(comment_elements)
    
        print(f"🧮 Loaded comments so far: {current_comment_count}")
    
        if current_comment_count == last_comment_count:
            same_height_attempts += 1
            print(f"⚠️ No new comments loaded... ({same_height_attempts}/{max_attempts})")
        else:
            print(f"✅ New comments loaded: +{current_comment_count - last_comment_count}")
            same_height_attempts = 0
            last_comment_count = current_comment_count
            num_of_scrolls += 1
    
        # Optional: fallback to scroll height comparison as backup
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            same_height_attempts += 1
        else:
            last_height = new_height

        
        is_scrape_comments = True
        if is_scrape_comments : # WOrk around - Set True early 
            MM_collector.update({'is_commented': True})

        print(f"Current number of scrolls {num_of_scrolls}")
        if num_of_scrolls == 0:
            print('We have scrolled to the bottom, but the height did not change!')
        
            try:
                # ✅ Wait for the comment section header to appear
                wait = WebDriverWait(driver, 10)
                clickable_element = wait.until(
                    EC.presence_of_element_located((By.XPATH, "//ytd-comments-header-renderer[contains(@class, 'style-scope')]"))
                )
                print("✅ Comment header detected.")
            except TimeoutException:
                print("⚠️ No comment header found — likely disabled comments or comments not loaded yet.")
                MM_collector.update({'is_commented': None})
                is_scrape_comments = False
                clickable_element = None
        
            if clickable_element:
                print("-----------COMMENTS-----------------------")
        
                # Print child elements to inspect structure
                all_yt_strings = clickable_element.find_elements(By.XPATH, ".//yt-formatted-string[contains(@class, 'count-text')]")
                for idx, e in enumerate(all_yt_strings):
                    print(f"[{idx}] -> '{e.text}' | class='{e.get_attribute('class')}' | id='{e.get_attribute('id')}'")
        
                # --- Detect comment count text ---
                hope_text_list = clickable_element.find_elements(
                    By.XPATH, ".//yt-formatted-string[contains(@class, 'count-text')]"
                )
        
                if hope_text_list:
                    for e_text in hope_text_list:
                        text_clean = e_text.text.strip().replace('\u202f', ' ').replace('\xa0', ' ')
                        print(f"🔍 Comment count text: '{text_clean}'")
        
                        match = re.search(r'^(\d+)\s*[^\w]*[Cc]omments', text_clean)
                        if match:
                            count = match.group(1)
                            print(f"✅ Matched comment count: {count}")
                            if count == '0':
                                print('🚫 Uncommented video')
                                MM_collector.update({'is_commented': False})
                                is_scrape_comments = False
                            else:
                                # Some reason not working 
                                print('💬 Commented video')
                                MM_collector.update({'is_commented': True})
                            break
                else:
                    print('❌ Disabled comments video')
                    MM_collector.update({'is_commented': None})
                    is_scrape_comments = False
                    
    
    COMMENT_DICT = []
    if is_scrape_comments:
        
        # Collect all loaded comments
        comments = driver.find_elements(*comments_path)
        
        for el in comments:
            emojies_string = []
            # Get any emojis
            emojies = el.find_elements(By.XPATH, ".//img[@alt]")
            for e_emoji in emojies:
                # print(e_emoji.get_attribute('alt'))
                actual_emoji = e_emoji.get_attribute('alt')
                emojies_string.append(actual_emoji)
            # Get the replies 
            replies = el.find_elements(*reply_comments_path)
            # print(replies.text)
            small_comment_dict = {}
            # Process the text
            li = el.text.split('\n')
            lit = list(filter(lambda x : not re.search(pattern=r"^Pinned", string=x), li))
        
            # Deal with unliked comments
            found_num = re.search(pattern=r'^\d{1,3}$', string=lit[-2]) # Checks if the second last element is a number
            if not found_num:
                print("Unliked comment")
                lit.insert(-1, '0')
            else:
                pass
            print(lit)
            poster, comment_age, *comment_text_content, like_count, reply = lit
            full_comment = ' '.join(comment_text_content)
            emoji_comment = ' '.join(emojies_string)
            full_comment += emoji_comment
            small_comment_dict.update({'poster_OP' : poster,
                                       'comment_age' : comment_age,
                                       'full_comment' : full_comment,
                                       'like_count' : like_count
                                      })
            COMMENT_DICT.append(small_comment_dict)
            # print("-----")
        # # Done can exit window
        driver.quit()
        return pd.DataFrame.from_dict(data=pd.Series(MM_collector)).T, pd.DataFrame(data=COMMENT_DICT)
    else:
        # # Done can exit window
        driver.quit()
        return pd.DataFrame.from_dict(data=pd.Series(MM_collector)).T, pd.DataFrame(data=COMMENT_DICT)
        
## Invoke 
# MM_df, comment_df = scrape_into_2_databases(url_ID='D_O6KXM7GwY') ## zOufUbKXjTY

## Data cleaning

In [6]:
from datetime import datetime, timedelta
import re
from datetime import timedelta
from dateutil import parser

def parse_relative_time(text):
    """
    Auxillary function 
    """
    # Match patterns like "25 minutes ago"
    match = re.match(r'(\d+)\s+(\w+)', text)
    if not match:
        return pd.NaT  # If format doesn't match, return missing
    
    value, unit = match.groups()
    value = int(value)
    unit = unit.lower()
    
    # Handle plural forms
    if unit.endswith('s'):
        unit = unit[:-1]
    
    # Convert to timedelta (approximate for months/years)
    if unit == 'year':
        return timedelta(days=value * 365)
    elif unit == 'month':
        return timedelta(days=value * 30)
    elif unit == 'week':
        return timedelta(weeks=value)
    elif unit == 'day':
        return timedelta(days=value)
    elif unit == 'hour':
        return timedelta(hours=value)
    elif unit == 'minute':
        return timedelta(minutes=value)
    elif unit == 'second':
        return timedelta(seconds=value)
    else:
        return pd.NaT

# invoke
parse_relative_time('25 minutes ago')

datetime.timedelta(seconds=1500)

In [7]:
## Data clea
def cast_datatypes(MM_df, comment_df):
    # print(f"is_commented_value : {is_commented}")
    try: # Outer try block: might encounters if video is private
        MM_df.replace({ 'views': { r'^([\d\,]+)\s(\w+)': r'\1' } }, regex=True, inplace=True)
        try: # Put app apply functions in try block to prevent errors on reruns
            # Views
            MM_df['views'] = MM_df['views'].apply(correct_number) 
            # Date
            MM_df['date'] =  MM_df['date'].apply(parse) #pd.to_datetime(MM_df['date'], format='%d %b %Y') 
            # MM_timestamp
            MM_df['MM_timestamp'] = MM_df['MM_timestamp'].apply(parse)
    
            if MM_df.loc[0, 'is_commented']:
                try: # There can be instances like 'rhPIKZgQVZo'
                    comment_df['like_count'] = comment_df['like_count'].astype(int)
                except KeyError:
                    pass
                pass
        except TypeError: # TypeError: expected string or bytes-like object
            pass
        finally:
            # Views
            MM_df['views'] = MM_df['views'].astype(int) # pd.to_numeric(MM_df['views'])
            # Likes
            MM_df['likes'] =  MM_df['likes'].astype(int)
        
        if MM_df.loc[0, 'is_commented']:
            try: # There can be instances 'rhPIKZgQVZo'
                comment_df['like_count'] = comment_df['like_count'].astype(int)
            except KeyError:
                MM_df.loc[0, 'is_commented'] = 'Type 1 Error'
                pass
        else:
            pass
    except ValueError:
        pass


    return MM_df, comment_df

In [8]:
# # # Invoke 
# # lOMrlMb611s
# a, b = scrape_into_2_databases(url_ID='D_O6KXM7GwY', is_print=True) #sIYNr_B1TEQ #QjXMxlR8zKo #3RKCWeOpip0 
# MM_df, comment_df = cast_datatypes(a, b)

## Attempt

In [9]:
# Reading the file - unpickling
import pickle
with open('./Data/overview_youtube_2.pkl', 'rb') as fp:
    overview_df = pickle.load(fp) # New and improved 
overview_df

,ID,mail,Scraped_DateTime,phone,language,privacy_public,info_outline,person_radar,my_videos,Videos_Not_Shorts,trending_up
0,@AFFIN,,2025-10-23 19:19:33.602173,,www.youtube.com/@AFFIN,Malaysia,Joined 18 Jun 2014,4.9K subscribers,325 videos,221,"15,050,294 views"
1,@AllianceBankMY,,2025-10-23 19:19:49.725717,,www.youtube.com/@AllianceBankMY,Malaysia,Joined 15 Aug 2013,8.89K subscribers,398 videos,356,"55,406,213 views"
2,@AmBankTV,,2025-10-23 19:20:05.871238,,www.youtube.com/@AmBankTV,Malaysia,Joined 1 Jul 2013,7.35K subscribers,432 videos,362,"27,976,034 views"
3,@bangkokbankchannel,,2025-10-23 19:20:22.260778,,www.youtube.com/@bangkokbankchannel,Thailand,Joined 17 Apr 2012,28.8K subscribers,124 videos,98,"8,970,431 views"
4,@BankofAmerica,,2025-10-23 19:20:38.180865,,www.youtube.com/@BankofAmerica,United States,Joined 18 Mar 2006,122K subscribers,169 videos,102,"788,249 views"
5,@BankofChinaManila,,2025-10-23 19:20:54.087877,,www.youtube.com/@BankofChinaManila,Philippines,Joined 19 Oct 2020,461 subscribers,8 videos,8,"38,112 views"
6,@labanquedunmondequichange,,2025-10-23 19:21:10.391375,,www.youtube.com/@labanquedunmondequichange,France,Joined 28 Jun 2006,57.9K subscribers,"2,815 videos",2579,"220,818,912 views"
7,@MyBoostApp,,2025-10-23 19:21:26.614858,,www.youtube.com/@MyBoostApp,Malaysia,Joined 15 Nov 2016,70.2K subscribers,611 videos,350,"106,143,812 views"
8,@CIMBGroupHoldingsBhd,,2025-10-23 19:22:00.576527,,www.youtube.com/@CIMBGroupHoldingsBhd,Malaysia,Joined 7 Sept 2010,67.6K subscribers,517 videos,468,"175,834,466 views"
9,@Citi,,2025-10-23 19:22:17.412780,,www.youtube.com/@Citi,,Joined 24 Oct 2005,94.2K subscribers,"1,680 videos",1663,"176,269,397 views"


In [10]:
# Convert the text data into Python list
# This data is a textfile of YT_Handle.pkl - if appeneded this YT_CHannel id done
memoized_channels = RW.read_txt_into_list('memo_database_scrape') # touch this file beforehand
memoized_IDs = RW.read_txt_into_list('memo_VideoID_scrape')
## Test one
import os
import re
from tqdm import tqdm
directory = r"./Data/Database/"
with open("./Data/memo_database_scrape.txt", "a") as writer: # touch this file beforehand
    for e_pickle in os.listdir(directory):
        if e_pickle not in memoized_channels:
            match = re.search(pattern=r"^(@\w+)\.pkl", string=e_pickle) # Match the Youtube_Handle; @AFFFIN, exclude hidden files
            if match:
                YT_handle = match.group(1)
                print(f'Starting the process for {YT_handle}')
                fullpath = os.path.normpath(os.path.join(directory, e_pickle))
                with open(fullpath, 'rb') as fp:
                    # Retrieve the dataframe - read it back
                    df = pickle.load(fp)
                    print(f"This {YT_handle}  has {df.shape[0]} video_IDs")


                    ## -------- Unpickle MEMO A ---------------
                    # Pick up where we left off 
                    if os.path.exists(fr'./Data/Lists/MM_temp/{YT_handle}.pkl'):
                        # Reading the file - unpickling
                        with open(fr'./Data/Lists/MM_temp/{YT_handle}.pkl', 'rb') as memo_1:
                            MM_BIG_LIST = pickle.load(memo_1)
                            print(f"Succesfully loaded pickle from : {f'./Data/Lists/MM_temp/{YT_handle}.pkl'}")
                    else:
                        MM_BIG_LIST = [] # To regather the data

                    if os.path.exists(fr'./Data/Lists/COMMENT_temp/{YT_handle}.pkl'):
                        # Reading the file - unpickling
                        with open(fr'./Data/Lists/COMMENT_temp/{YT_handle}.pkl', 'rb') as memo_2:
                            COMMENT_BIG_LIST = pickle.load(memo_2)
                            print(f"Succesfully loaded pickle from : {f'./Data/Lists/COMMENT_temp/{YT_handle}.pkl'}")
                    else:
                        COMMENT_BIG_LIST = [] # To regather the data
                    
                    ## -------- Unpickle MEMO A ---------------

                    progress_count = 0
                    with open("./Data/memo_VideoID_scrape.txt", "a") as small_writer:
                        for idx, row_dict in tqdm(df.to_dict(orient='index').items()):
                            print(f"Currently scraping for {row_dict['Youtube_Video_IDs']}")
                            if row_dict['Youtube_Video_IDs'] not in memoized_IDs:
                                ## Invoke the function to generate 2 dataframes
                                a, b = scrape_into_2_databases(url_ID=row_dict['Youtube_Video_IDs'])
                                MM_df_oneline, COMMENT_df = cast_datatypes(a, b)
                                # # Good so far, we will need to add the primary keys back
                                # display(MM_df_oneline)
                                # display(COMMENT_df)
                
                                # Insert the two series one after the other.
                                MM_df_oneline.insert(loc=0, column='Youtube_Handle', value=row_dict['Youtube_Handle'])
                                MM_df_oneline.insert(loc=0, column='Youtube_Video_IDs', value=row_dict['Youtube_Video_IDs'])
                
                                # Just the ID for the comment_df
                                COMMENT_df.insert(loc=0, column='Youtube_Handle', value=row_dict['Youtube_Handle'])
                                COMMENT_df.insert(loc=0, column='Youtube_Video_IDs', value=row_dict['Youtube_Video_IDs'])
                            
                
                                MM_dict = MM_df_oneline.to_dict(orient='index') # One solitary index
                                COMMENT_dict = COMMENT_df.to_dict(orient='index')
                
                                # Good so far, we will need to add the primary keys back
                                display(MM_df_oneline)
                                display(COMMENT_df)
                                
                                # Write in-disk, across sessions
                                small_writer.write(f"{row_dict['Youtube_Video_IDs']}\n")
                                small_writer.flush()
                                # Write in-memory, within the same session
                                memoized_IDs.append(row_dict['Youtube_Video_IDs'])
                                
                                MM_BIG_LIST.append(MM_dict)
                                COMMENT_BIG_LIST.append(COMMENT_dict)


                                ## -------- Pickle MEMO A ---------------
                                print(f"Saving...")
                                ## Save progrss
                                # Write into a file everytime
                                with open(fr'./Data/Lists/MM_temp/{YT_handle}.pkl', 'wb') as save_1:
                                    pickle.dump(MM_BIG_LIST, save_1)
                                with open(fr'./Data/Lists/COMMENT_temp/{YT_handle}.pkl', 'wb') as save_2:
                                    pickle.dump(COMMENT_BIG_LIST, save_2)
                                ## -------- Pickle MEMO A ---------------
                                 
                                progress_count += 1
                            else:
                                print(f"Skiiping {row_dict['Youtube_Video_IDs']}, already memoized")
                                progress_count += 1
                            print(f"Done {progress_count}/{df.shape[0]}")
        
                    ## Post-processing
                    print(f'Performing post-processing for {YT_handle}')
                    LIST = []
                    for subdict in MM_BIG_LIST:
                        for idx, subdict_2 in subdict.items():
                            LIST.append(subdict_2)
                    MM_dataframe = pd.DataFrame.from_dict(LIST)
        
                    LIST2 = []
                    for subdict in COMMENT_BIG_LIST:
                        for idx, subdict_2 in subdict.items():
                            LIST2.append(subdict_2)
                    COMMENT_dataframe = pd.DataFrame.from_dict(LIST2)
                    
                    ## Write MM_table 
                    with open(fr'./Data/MM_database/{match.group(1)}.pkl', 'wb') as file_1:
                        pickle.dump(MM_dataframe, file_1)
                    ## Write COmment_table 
                    with open(fr'./Data/COMMENT_database/{match.group(1)}.pkl', 'wb') as file_2:
                        pickle.dump(COMMENT_dataframe, file_2) 

                    # Write in-disk, across sessions
                    writer.write(f"{e_pickle}\n")
                    writer.flush()
                    # Write in-memory, within the same session
                    memoized_channels.append(e_pickle)
                    
        else:
            print(f"Skiiping {e_pickle}, already memoized")
            pass            

Skiiping @channelocbc.pkl, already memoized
Skiiping @AllianceBankMY.pkl, already memoized
Skiiping @labanquedunmondequichange.pkl, already memoized
Skiiping @PublicBankGroup.pkl, already memoized
Skiiping @CIMBGroupHoldingsBhd.pkl, already memoized
Skiiping @smbcgroup.pkl, already memoized
Skiiping @BankofChinaManila.pkl, already memoized
Skiiping @GXBank.pkl, already memoized
Skiiping @Citi.pkl, already memoized
Skiiping @HSBC_MY.pkl, already memoized
Skiiping @standardchartered.pkl, already memoized
Skiiping @rhbgroup.pkl, already memoized
Skiiping @jpmorgan.pkl, already memoized
Skiiping @MizuhoAmericas.pkl, already memoized
Skiiping @AmBankTV.pkl, already memoized
Skiiping @BankofAmerica.pkl, already memoized
Skiiping @MyBoostApp.pkl, already memoized
Skiiping @DeutscheBank.pkl, already memoized
Skiiping @AFFIN.pkl, already memoized
Skiiping @maybankvideos.pkl, already memoized
Skiiping @hongleongbankmy.pkl, already memoized
Skiiping @bangkokbankchannel.pkl, already memoized
Skiip

## Pickle Collate

In [11]:
## Read back data
# Takes a while to read the pickle
import pickle
# Reading the file - unpickling

# Pure MM and Contents
MM_dir = r'./Data/MM_database/'
MM_contents = os.listdir(r'./Data/MM_database/')
COMMENT_dir = r'./Data/COMMENT_database/'
COMMENT_contents = os.listdir(r'./Data/COMMENT_database/')

## Temp folders
MM_temp_dir = fr'./Data/Lists/MM_temp/'
MM_temp_contents = os.listdir(MM_temp_dir)
COMMENT_temp_dir = fr"./Data/Lists/COMMENT_temp/"
COMMENT_temp_contents = os.listdir(COMMENT_temp_dir)

print(MM_temp_contents)

COLLATE_MM = pd.DataFrame()
COLLATE_COMMENT = pd.DataFrame()
for file in MM_temp_contents:
    match = re.search(pattern=r"^[^\.]", string=file)
    if match:
        print('Metadata:\n')
        with open(os.path.normpath(os.path.join(MM_temp_dir, file)), 'rb') as fp:
            MM_BIG_LIST = pickle.load(fp) # e_MM_df or MM_BIG_LIST
            # display(e_MM_df) 
            LIST = []
            for subdict in MM_BIG_LIST:
                for idx, subdict_2 in subdict.items():
                    LIST.append(subdict_2)
            MM_dataframe = pd.DataFrame.from_dict(LIST)
            COLLATE_MM = pd.concat([COLLATE_MM, MM_dataframe], axis='index')
            display(pd.concat([MM_dataframe.head(), MM_dataframe.tail()]))

            
            
        print('Comments:\n')
        print(COMMENT_contents)
        with open(os.path.normpath(os.path.join(COMMENT_temp_dir, file)), 'rb') as fp_2:
            COMMENT_BIG_LIST = pickle.load(fp_2)  # e_COMMENT_df or COMMENT_BIG_LIST
            # display(e_COMMENT_df)
            LIST2 = []
            for subdict in COMMENT_BIG_LIST:
                for idx, subdict_2 in subdict.items():
                    LIST2.append(subdict_2)
            COMMENT_dataframe = pd.DataFrame.from_dict(LIST2)
            COLLATE_COMMENT = pd.concat([COLLATE_COMMENT, COMMENT_dataframe], axis='index')
            display(pd.concat([COMMENT_dataframe.head(), COMMENT_dataframe.tail()]))


COLLATE_MM.reset_index(inplace=True, drop=True)
COLLATE_COMMENT.reset_index(inplace=True, drop=True)

# Write into COLLATE folder
with open('./Data/COLLATE/MM.pkl', 'wb') as f:
    pickle.dump(COLLATE_MM, f)
with open('./Data/COLLATE/COMMENT.pkl', 'wb') as f:
    pickle.dump(COLLATE_COMMENT, f)

['@channelocbc.pkl', '@AllianceBankMY.pkl', '@labanquedunmondequichange.pkl', '@PublicBankGroup.pkl', '@CIMBGroupHoldingsBhd.pkl', '@smbcgroup.pkl', '@BankofChinaManila.pkl', '@GXBank.pkl', '@Citi.pkl', '@HSBC_MY.pkl', '@standardchartered.pkl', '@rhbgroup.pkl', '@jpmorgan.pkl', '@MizuhoAmericas.pkl', '@AmBankTV.pkl', '@BankofAmerica.pkl', '@MyBoostApp.pkl', '@DeutscheBank.pkl', '@AFFIN.pkl', '@maybankvideos.pkl', '@hongleongbankmy.pkl', '.ipynb_checkpoints', '@bangkokbankchannel.pkl', '@uob.pkl', '@MUFGBankChannel.pkl']
Metadata:



,Youtube_Video_IDs,Youtube_Handle,video_length,views,date,title,likes,description,MM_timestamp,is_commented
0,LiFxYCLSvb4&pp=0gcJCQYKAYcqIYzv,@channelocbc,1:07,90,2025-10-15,OCBC Structured Notes: Customise Your Risk and...,3,Enjoy the flexibility of attaining exposure to...,2025-11-02 22:59:13,False
1,N9SBRNEAnhI,@channelocbc,2:11,514,2025-10-10,OCBC - Banking on Us - Episode 1,16,"While the city sleeps, our teams are wide awak...",2025-11-02 22:59:33,False
2,PLqT8W6JNDY,@channelocbc,1:47,104,2025-10-03,OCBC One Connect - Integrated Ecosystem Suppor...,1,OCBC One Connect - Integrated Ecosystem Suppor...,2025-11-02 22:59:52,False
3,4UBrl92xXOU,@channelocbc,1:59,79932,2025-10-01,Behind the scenes of our 2025 brand campaign –...,7,Watch how our latest brand campaign stories ge...,2025-11-02 23:00:12,False
4,cYtlA9Q7UAo,@channelocbc,4:18,178,2025-09-30,OCBC華僑銀行市場熱點: 預期今年減息三次 (22-09-2025),2,OCBC華僑銀行財富管理專家團隊預期，美國就業數據疲弱，將促使聯儲局加快減息，預期聯儲局將於...,2025-11-02 23:00:32,False
819,x77sEC98AxI,@channelocbc,1:35,1073,2018-04-17,Gastronomic Adventures featuring Sushi Kimura ...,3,"At Sushi Kimura, expect to enjoy seasonally cu...",2025-11-07 02:15:41,False
820,30w8gUkZEoQ,@channelocbc,3:49,126,2018-04-10,Project Training Wheels S3 E1: On your bikes...,0,A new group of 16 non-cyclists attempt to lear...,2025-11-07 02:15:58,False
821,7gg3e7mlWcM,@channelocbc,1:50,689,2018-04-09,Campus Star Awards 2018: Recognising Our Inter...,0,"Each year, Campus At OCBC appreciates our inte...",2025-11-07 02:16:14,True
822,q81Zpm5oDNQ,@channelocbc,0:20,702,2018-04-09,Bank with your voice today!,1,OCBC Bank is the first in Singapore to let you...,2025-11-07 02:16:33,False
823,AqrZxrIqsJM,@channelocbc,1:14,1725,2018-04-03,Game Changers at OCBC | Group Operations & Tec...,4,"Within our team, it’s the people that make the...",2025-11-07 02:16:50,False


Comments:

['@channelocbc.pkl', '@AllianceBankMY.pkl', '@labanquedunmondequichange.pkl', '@PublicBankGroup.pkl', '@CIMBGroupHoldingsBhd.pkl', '@smbcgroup.pkl', '@BankofChinaManila.pkl', '@GXBank.pkl', '@Citi.pkl', '@HSBC_MY.pkl', '@standardchartered.pkl', '@rhbgroup.pkl', '@jpmorgan.pkl', '@MizuhoAmericas.pkl', '@AmBankTV.pkl', '@BankofAmerica.pkl', '@MyBoostApp.pkl', '@DeutscheBank.pkl', '@AFFIN.pkl', '@maybankvideos.pkl', '@hongleongbankmy.pkl', '.ipynb_checkpoints', '@bangkokbankchannel.pkl', '@uob.pkl', '@MUFGBankChannel.pkl']


,Youtube_Video_IDs,poster_OP,comment_age,full_comment,like_count,Youtube_Handle
0,FmQz15WJkWE,@tinkerlee-r9z,3 weeks ago,BECAREFUL WHAT YOU WISH FOR - CIRCUS ???!...,0,NaN
1,zb3oKcLv1yA,@kayanemahberkah,1 month ago,kak cara pindahin ocbc mobile dari hp lama ke ...,0,NaN
2,Y6GQJUyjDO4,@VikkiNesspara-f2u,1 month ago,happy 60th birthday Singapore 🇸🇬🇸🇬🇸🇬️🇸🇬️️🇸🇬️🇸🇬...,0,NaN
3,GGbcl2AcSBU,@RisNandar-c9h,2 months ago,OCBC ❤ ❤,0,NaN
4,meamip7zfzI,@BozuBoyYT,11 days ago,The person doing the voice over is so good!,2,NaN
323,mpbVv4swkzs&pp=0gcJCQYKAYcqIYzv,@jaketan5172,3 years ago,Most value-adding contrarian and yet not liste...,0,@channelocbc
324,avodcx-LQpo,@zueanzuean2330,4 years ago,After transfer why didn't have receipt,0,@channelocbc
325,-9TXIjL-hXo,@kokbontan206,2 years ago,请问我以前放在银行的存款还能拿回来吗？,0,@channelocbc
326,NPq2Tr5J76E,@gladeglades6388,1 year ago,If OCBC considered presenting these 2 in publi...,0,@channelocbc
327,7gg3e7mlWcM,@ngankhayseng7905,7 years ago,0:33 (This is a brain) OCBC: This is a brain m...,0,@channelocbc


Metadata:



...

,Youtube_Video_IDs,Youtube_Handle,poster_OP,comment_age,full_comment,like_count
0,D-ovCsB7Y50,@uob,@Moon-iv1xy,7 days ago,so excited for the new album!!,0
1,D-ovCsB7Y50,@uob,@Misnomer_alan,9 days ago,❤ ❤ ❤ ❤,0
2,cDMfGtU6ROc&pp=0gcJCQYKAYcqIYzv,@uob,@linarrow1275,2 weeks ago,A song that will never get tiresome,0
3,cDMfGtU6ROc&pp=0gcJCQYKAYcqIYzv,@uob,@JunyaoZhu-g1d,2 weeks ago,So healing,0
4,cDMfGtU6ROc&pp=0gcJCQYKAYcqIYzv,@uob,@Misnomer_alan,2 weeks ago,Love this ❤ ❤ ❤,0
525,4tA8wtLaui4,@uob,@lamanblog,9 years ago (edited),Mind anyone can share what song is this (Remem...,0
526,4tA8wtLaui4,@uob,@blakesmith7151,7 years ago,Fiiirrrsstt,0
527,4tA8wtLaui4,@uob,@jonaerakua1,8 years ago,"Thing is, you can inherit dad's skin AND mom's...",0
528,4tA8wtLaui4,@uob,@JokeRQuas,7 years ago,.,0
529,gA0hQtE6TL4,@uob,@floristdahlins8520,9 years ago,",",0


Metadata:



,Youtube_Video_IDs,Youtube_Handle,video_length,views,date,title,likes,description,MM_timestamp,is_commented
0,JQ4mfx8jFQ4,@MUFGBankChannel,2:18,426.0,2025-10-23,「三菱ＵＦＪ銀行」アプリ操作方法 カード・通帳・印鑑の紛失や再発行手続きはアプリで簡単【三菱...,0.0,スマートフォンアプリ「三菱ＵＦＪ銀行」でキャッシュカードや通帳、印鑑に関する紛失のお届けや再...,2025-11-03 15:57:15,None
1,dBFpxf79fs0,@MUFGBankChannel,1:12,1641.0,2025-07-06,残高証明書はPDFならアプリで即時発行＆手数料無料！【三菱UFJ銀行公式】,0.0,☆この商品のHPやキャンペーン情報☆\nスマートフォンアプリ「かんたん手続アプリ」 \n残高...,2025-11-03 15:57:44,None
2,XQspWed5lsY,@MUFGBankChannel,2:00,3439.0,2025-06-15,【エムット】コンセプトMOVIE 【三菱UFJ銀行公式】,0.0,お金のあれこれ、まるっと。「エムット」\n\nお金をつかう、ためる、ふやす、そして次世代に繋...,2025-11-03 15:58:15,None
3,HMWHmNmM0TI,@MUFGBankChannel,0:15,12733.0,2025-06-15,「アプリ1つでアクセス」篇【三菱UFJ銀行公式】,0.0,三菱UFJ銀行なら\nアプリ1つでいろんな金融サービスに！\n\n※本編後半に表示の「新規口...,2025-11-03 15:58:45,None
4,3PeHEunv8Ts,@MUFGBankChannel,2:41,2035.0,2025-06-02,NISA制度のポイント【三菱UFJ銀行公式】,0.0,2024年1月より、NISA制度はより魅力的な制度に変わりました。\n本動画では、NISA制...,2025-11-03 15:59:16,None
91,3MmwCVxXWks&pp=0gcJCQYKAYcqIYzv,@MUFGBankChannel,1:26,3658.0,2020-12-27,『やりたいことがあります』サステイナブルファイナンス編【三菱UFJ銀行公式】,0.0,↓↓行員達の「私たちのやりたいこと」を新卒採用ホームページに記載しておりますので是非ご覧くだ...,2025-11-03 16:46:58,None
92,xhX81v6p64k,@MUFGBankChannel,1:37,8666.0,2020-12-27,『やりたいことがあります』プロジェクトファイナンス編【三菱UFJ銀行公式】,0.0,↓↓行員達の「私たちのやりたいこと」を新卒採用ホームページに記載しておりますので是非ご覧くだ...,2025-11-03 16:47:31,None
93,SPUo_4hOa5w,@MUFGBankChannel,1:33,3271.0,2020-12-27,『やりたいことがあります』成長産業支援室編【三菱UFJ銀行公式】,0.0,↓↓行員達の「私たちのやりたいこと」を新卒採用ホームページに記載しておりますので是非ご覧くだ...,2025-11-03 16:48:02,None
94,Fll6MebIf8I,@MUFGBankChannel,2:30,6420.0,2020-03-30,「遺産整理業務【わかち愛】 ご紹介」篇,0.0,No description has been added to this video,2025-11-03 16:48:31,None
95,tew6Ly8NDcc,@MUFGBankChannel,4:50,3859.0,2019-08-12,動画で学ぶ資産運用「つみたて投資によろしく」,0.0,No description has been added to this video,2025-11-03 16:49:01,None


Comments:

['@channelocbc.pkl', '@AllianceBankMY.pkl', '@labanquedunmondequichange.pkl', '@PublicBankGroup.pkl', '@CIMBGroupHoldingsBhd.pkl', '@smbcgroup.pkl', '@BankofChinaManila.pkl', '@GXBank.pkl', '@Citi.pkl', '@HSBC_MY.pkl', '@standardchartered.pkl', '@rhbgroup.pkl', '@jpmorgan.pkl', '@MizuhoAmericas.pkl', '@AmBankTV.pkl', '@BankofAmerica.pkl', '@MyBoostApp.pkl', '@DeutscheBank.pkl', '@AFFIN.pkl', '@maybankvideos.pkl', '@hongleongbankmy.pkl', '.ipynb_checkpoints', '@bangkokbankchannel.pkl', '@uob.pkl', '@MUFGBankChannel.pkl']


""


In [12]:
COLLATE_MM.groupby('Youtube_Handle')['Youtube_Handle'].count()

Youtube_Handle
@AFFIN                         221
@AllianceBankMY                356
@AmBankTV                      362
@BankofAmerica                 102
@BankofChinaManila               8
@CIMBGroupHoldingsBhd          468
@Citi                         1663
@DeutscheBank                 1919
@GXBank                         33
@HSBC_MY                       188
@MUFGBankChannel                96
@MizuhoAmericas                154
@MyBoostApp                    350
@PublicBankGroup               206
@bangkokbankchannel             98
@channelocbc                   824
@hongleongbankmy               197
@jpmorgan                     1282
@labanquedunmondequichange    2579
@maybankvideos                1348
@rhbgroup                      561
@smbcgroup                      21
@standardchartered            1663
@uob                          1210
Name: Youtube_Handle, dtype: int64

In [13]:
COLLATE_COMMENT[COLLATE_COMMENT.Youtube_Handle.isna()]

,Youtube_Video_IDs,poster_OP,comment_age,full_comment,like_count,Youtube_Handle
0,FmQz15WJkWE,@tinkerlee-r9z,3 weeks ago,BECAREFUL WHAT YOU WISH FOR - CIRCUS ???!...,0,NaN
1,zb3oKcLv1yA,@kayanemahberkah,1 month ago,kak cara pindahin ocbc mobile dari hp lama ke ...,0,NaN
2,Y6GQJUyjDO4,@VikkiNesspara-f2u,1 month ago,happy 60th birthday Singapore 🇸🇬🇸🇬🇸🇬️🇸🇬️️🇸🇬️🇸🇬...,0,NaN
3,GGbcl2AcSBU,@RisNandar-c9h,2 months ago,OCBC ❤ ❤,0,NaN
4,meamip7zfzI,@BozuBoyYT,11 days ago,The person doing the voice over is so good!,2,NaN
5,j-UzkGGiAGg,@RisNandar-c9h,2 months ago,OCBC ❤ ❤,0,NaN
6,KsrYGzbc2GA,@RisNandar-c9h,3 months ago,OCBC ❤ ❤,0,NaN
7,gwrlQkXp7qk,@dc1211,3 months ago,This is touching and inspiring.,0,NaN
8,j7JD5Qp-c5c,@RisNandar-c9h,4 months ago,OCBC ❤ ❤,1,NaN
9,aqpc1usDDsY,@RoyShouriMustango,4 months ago,No one cares,0,NaN


In [14]:
with open('./Data/COLLATE/COMMENT.pkl', 'wb') as f:
    pickle.dump(COLLATE_COMMENT, f)

```python
import os
import re
import pickle
import pandas as pd
from tqdm import tqdm
import pyarrow as pa
import pyarrow.parquet as pq

# --- Your memoization text-based files ---
memoized_channels = RW.read_txt_into_list('memo_database_scrape')
memoized_IDs = RW.read_txt_into_list('memo_VideoID_scrape')

directory = r"./Data/Database/"

def append_to_parquet(df, path):
    """Append or create parquet file efficiently."""
    table = pa.Table.from_pandas(df)
    if os.path.exists(path):
        with pq.ParquetWriter(path, table.schema, compression='snappy', use_dictionary=True) as writer:
            writer.write_table(table)
    else:
        pq.write_table(table, path, compression='snappy', use_dictionary=True)


with open("./Data/memo_database_scrape.txt", "a") as writer:
    for e_pickle in os.listdir(directory):
        if e_pickle not in memoized_channels:
            match = re.search(r"^(@\w+)\.parquet", e_pickle)
            if not match:
                continue

            YT_handle = match.group(1)
            print(f"Starting the process for {YT_handle}")
            fullpath = os.path.join(directory, match.group(0))

            df = pd.read_parquet(fullpath)
            print(f"{YT_handle} has {df.shape[0]} video_IDs")

            # --- Define parquet paths ---
            MM_temp_path = f'./Data/Lists/MM_temp/{YT_handle}.parquet'
            COMMENT_temp_path = f'./Data/Lists/COMMENT_temp/{YT_handle}.parquet'

            with open("./Data/memo_VideoID_scrape.txt", "a") as small_writer:
                for idx, row_dict in tqdm(df[:240].to_dict(orient='index').items()):
                    vid_id = row_dict['Youtube_Video_IDs']
                    if vid_id in memoized_IDs:
                        print(f"Skipping {vid_id}, already memoized")
                        continue

                    print(f"Currently scraping for {vid_id}")
                    a, b = scrape_into_2_databases(url_ID=vid_id)
                    MM_df_oneline, COMMENT_df = cast_datatypes(a, b)
                    
                    # Add identifying columns
                    MM_df_oneline.insert(0, 'Youtube_Handle', YT_handle)
                    MM_df_oneline.insert(0, 'Youtube_Video_IDs', vid_id)
                    COMMENT_df.insert(0, 'Youtube_Video_IDs', vid_id)
                
                    display(MM_df_oneline)
                    display(COMMENT_df)

                    # --- Append to temp parquet ---
                    append_to_parquet(MM_df_oneline, MM_temp_path)
                    append_to_parquet(COMMENT_df, COMMENT_temp_path)

                    # Update memo
                    small_writer.write(f"{vid_id}\n")
                    small_writer.flush()
                    memoized_IDs.append(vid_id)

            # --- Post-processing final merge ---
            print(f"Performing post-processing for {YT_handle}")

            # Concatenate temp parquet into final database folder
            final_MM_path = f'./Data/MM_database/{YT_handle}.parquet'
            final_COMMENT_path = f'./Data/COMMENT_database/{YT_handle}.parquet'

            # Merge existing temp parquet files into a clean final version
            if os.path.exists(MM_temp_path):
                temp_MM_df = pd.read_parquet(MM_temp_path)
                # temp_MM_df['is_commented'] = temp_MM_df['is_commented'].astype(str)
                temp_MM_df.to_parquet(final_MM_path, compression='snappy', index=False)
            if os.path.exists(COMMENT_temp_path):
                temp_COMMENT_df = pd.read_parquet(COMMENT_temp_path)
                temp_COMMENT_df.to_parquet(final_COMMENT_path, compression='snappy', index=False)

            # Update channel memo
            writer.write(f"{e_pickle}\n")
            writer.flush()
            memoized_channels.append(e_pickle)

        else:
            print(f"Skipping {e_pickle}, already memoized")
```

## Migration script

In [122]:
import pandas as pd
import pickle
from pathlib import Path
def safe_type_coercion(df):
    for col in df.columns:
        series = df[col]
        # Skip columns that are already datetimes
        if pd.api.types.is_datetime64_any_dtype(series):
            continue
        
        # Try detecting datetime-like object columns (as strings)
        if series.dtype == object:
            try:
                parsed = pd.to_datetime(series, errors='raise', utc=False)
                # Only treat as datetime if a large majority parsed successfully
                if parsed.notna().mean() > 0.9:
                    df[col] = parsed
                    continue
            except Exception:
                pass

        # Try numeric conversion
        try:
            df[col] = pd.to_numeric(series)
            continue
        except Exception:
            pass

        # Fallback: convert to string
        try:
            df[col] = series.astype(str)
        except Exception:
            pass
    
    return df

def convert_pickle_to_parquet(pkl_path):
    try:
        with open(pkl_path, "rb") as f:
            data = pickle.load(f)
    except Exception as e:
        print(f"❌ Failed to load {pkl_path}: {e}")
        return

    parquet_path = Path(pkl_path).with_suffix(".parquet")

    # --- Normalize into DataFrame ---
    if isinstance(data, pd.DataFrame):
        df = data
    elif isinstance(data, list):
        if len(data) > 0 and isinstance(data[0], dict):
            df = pd.DataFrame(data)
        else:
            df = pd.DataFrame({"values": data})
    elif isinstance(data, dict):
        df = pd.DataFrame.from_dict(data, orient="index")
    else:
        print(f"⚠️ Skipping {pkl_path} — not a tabular structure.")
        return

    # --- Coerce all columns to strings to avoid pyarrow dtype issues ---
    df = safe_type_coercion(df)

    # --- Write Parquet safely ---
    try:
        df.to_parquet(parquet_path, compression="snappy", index=False)
        print(f"✅ Converted {pkl_path} → {parquet_path}")
    except Exception as e:
        print(f"❌ Failed to write parquet for {pkl_path}: {e}")

# --- MAIN LOOP ---
for root in root_dirs:
    for file in os.listdir(root):
        if file.endswith(".pkl"):
            fullpath = os.path.join(root, file)
            convert_pickle_to_parquet(fullpath)

/var/folders/gd/ww733_552m32hsvqk2bnrwqh0000gp/T/ipykernel_38467/2503055579.py:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parsed = pd.to_datetime(series, errors='raise', utc=False)
/var/folders/gd/ww733_552m32hsvqk2bnrwqh0000gp/T/ipykernel_38467/2503055579.py:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parsed = pd.to_datetime(series, errors='raise', utc=False)
/var/folders/gd/ww733_552m32hsvqk2bnrwqh0000gp/T/ipykernel_38467/2503055579.py:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parsed = pd.to_datetime(series, errors='raise', utc=False)
/var/folders/gd/

✅ Converted ./Data/Database/@channelocbc.pkl → Data/Database/@channelocbc.parquet
✅ Converted ./Data/Database/@AllianceBankMY.pkl → Data/Database/@AllianceBankMY.parquet
✅ Converted ./Data/Database/@labanquedunmondequichange.pkl → Data/Database/@labanquedunmondequichange.parquet
✅ Converted ./Data/Database/@PublicBankGroup.pkl → Data/Database/@PublicBankGroup.parquet
✅ Converted ./Data/Database/@CIMBGroupHoldingsBhd.pkl → Data/Database/@CIMBGroupHoldingsBhd.parquet
✅ Converted ./Data/Database/@smbcgroup.pkl → Data/Database/@smbcgroup.parquet
✅ Converted ./Data/Database/@BankofChinaManila.pkl → Data/Database/@BankofChinaManila.parquet
✅ Converted ./Data/Database/@GXBank.pkl → Data/Database/@GXBank.parquet
✅ Converted ./Data/Database/@Citi.pkl → Data/Database/@Citi.parquet
✅ Converted ./Data/Database/@HSBC_MY.pkl → Data/Database/@HSBC_MY.parquet
✅ Converted ./Data/Database/@standardchartered.pkl → Data/Database/@standardchartered.parquet
✅ Converted ./Data/Database/@rhbgroup.pkl → Data/D

/var/folders/gd/ww733_552m32hsvqk2bnrwqh0000gp/T/ipykernel_38467/2503055579.py:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parsed = pd.to_datetime(series, errors='raise', utc=False)
/var/folders/gd/ww733_552m32hsvqk2bnrwqh0000gp/T/ipykernel_38467/2503055579.py:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parsed = pd.to_datetime(series, errors='raise', utc=False)
/var/folders/gd/ww733_552m32hsvqk2bnrwqh0000gp/T/ipykernel_38467/2503055579.py:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parsed = pd.to_datetime(series, errors='raise', utc=False)
/var/folders/gd/

## Sanity check - migration successful

In [72]:
import pandas as pd

# df_old = pickle.load(open("./Data/MM_database/@AFFIN.pkl", "rb"))
df_new = pd.read_parquet("./Data/MM_database/@AFFIN.parquet")

list_temp_MM_df = pd.read_parquet(r"./Data/Lists/MM_temp/@AFFIN.parquet")
list_temp_MM_df
# print(df_.columns.symmetric_difference(df_new.columns))

0      {'Youtube_Video_IDs': 'RMZdBov0FrQ', 'Youtube_...
1      {'Youtube_Video_IDs': 'Hhi_ERnrv1g', 'Youtube_...
2      {'Youtube_Video_IDs': 't8UCJJafTd0', 'Youtube_...
3      {'Youtube_Video_IDs': 'PyuTTP4C12c', 'Youtube_...
4      {'Youtube_Video_IDs': 'SPZ4q1CnxnE', 'Youtube_...
                             ...                        
216    {'Youtube_Video_IDs': 'raaESaRta4k', 'Youtube_...
217    {'Youtube_Video_IDs': 'UyWEmdaX2EI', 'Youtube_...
218    {'Youtube_Video_IDs': 'PjBmn4J4z4E', 'Youtube_...
219    {'Youtube_Video_IDs': 'P2d9kJ8pBLE', 'Youtube_...
220    {'Youtube_Video_IDs': 'mVr-5QEAlKg', 'Youtube_...
Name: 0, Length: 221, dtype: object

In [54]:
df_new

,Youtube_Video_IDs,Youtube_Handle,video_length,views,date,title,likes,description,MM_timestamp,is_commented
0,RMZdBov0FrQ,@AFFIN,1:00,220,2025-08-29,"AFFIN Borneo Credit Card/-i - Unlimited, the B...",5,"🌴✨ Hello Sabah & Sarawak!\nYour card, your per...",2025-10-30 18:08:36,False
1,Hhi_ERnrv1g,@AFFIN,1:17,127,2025-08-29,Detik Niaga TV3 : Kerjasama Strategik antara A...,3,"A memorable event with Marriott International,...",2025-10-30 18:08:54,False
2,t8UCJJafTd0,@AFFIN,0:52,111,2025-08-26,ASTON MARTIN X AFFIN,1,The road is yours 🚘✨\n\nEnjoy exclusive owners...,2025-10-30 18:09:10,False
3,PyuTTP4C12c,@AFFIN,2:22,85,2025-08-13,Event Highlight AFFIN BizChat KL 2025,3,AFFIN SME BizChat 2025 in Kuala Lumpur was an ...,2025-10-30 18:09:28,False
4,SPZ4q1CnxnE,@AFFIN,1:14,97,2025-07-17,AFFIN SME BizChat KL 2025 - Teaser,10,"The highly anticipated flagship business talk,...",2025-10-30 18:09:47,True
...,...,...,...,...,...,...,...,...,...,...
216,raaESaRta4k,@AFFIN,0:31,126,2021-02-26,Photo Challenge Contest,1,Don’t miss this opportunity! Campaign starts n...,2025-11-01 01:26:26,False
217,UyWEmdaX2EI,@AFFIN,0:30,444,2017-01-12,AFFINBANK Group - Karnival Kewangan 2017,3,Karnival Kewangan 2017 at PWTC,2025-11-01 01:26:42,False
218,PjBmn4J4z4E,@AFFIN,0:30,95,2017-01-10,FINAL KARNIVAL KEWANGAN BM,0,Karnival Kewangan 2017\n\nKarnival yang bertem...,2025-11-01 01:26:59,False
219,P2d9kJ8pBLE,@AFFIN,0:30,229,2017-01-09,KARNIVAL KEWANGAN 2017,2,Karnival yang bertemakan “Utamakan Keperluan K...,2025-11-01 01:27:15,False


## Parquet Collate

In [15]:
## Temp folders
MM_temp_dir = fr'./Data/Lists/MM_temp/'
MM_temp_contents = os.listdir(MM_temp_dir)
COMMENT_temp_dir = fr"./Data/Lists/COMMENT_temp/"
COMMENT_temp_contents = os.listdir(COMMENT_temp_dir)

# Pure MM and Contents
MM_dir = r'./Data/MM_database/'
MM_contents = os.listdir(r'./Data/MM_database/')
COMMENT_dir = r'./Data/COMMENT_database/'
COMMENT_contents = os.listdir(r'./Data/COMMENT_database/')


print(MM_contents)

COLLATE_MM = pd.DataFrame()
COLLATE_COMMENT = pd.DataFrame()
for file in MM_contents:
    match = re.search(pattern=r"^[^\.].*\.parquet$", string=file)
   
    if match:
        print('Metadata:\n')
        # Read the Parquet file into a pandas DataFrame
        MM_df = pd.read_parquet(os.path.normpath(os.path.join(MM_dir, match.group(0))))
        COLLATE_MM = pd.concat([COLLATE_MM, MM_df], axis='index')
        display(pd.concat([MM_df.head(), MM_df.tail()]))

        print('Comments:\n')
        COMMENT_df = pd.read_parquet(os.path.normpath(os.path.join(COMMENT_dir, match.group(0))))
        COLLATE_COMMENT = pd.concat([COLLATE_COMMENT, COMMENT_df], axis='index')
        display(pd.concat([COMMENT_df.head(), COMMENT_df.tail()]))


COLLATE_MM.reset_index(inplace=True, drop=True)
COLLATE_MM['is_commented'] = COLLATE_MM['is_commented'].astype(str)
COLLATE_MM['video_length'] = COLLATE_MM['video_length'].astype(str)
COLLATE_COMMENT.reset_index(inplace=True, drop=True)

# Write into COLLATE folder
COLLATE_MM.to_parquet(r"./Data/COLLATE/MM.parquet", compression="snappy", index=False)
COLLATE_COMMENT.to_parquet(r"./Data/COLLATE/COMMENT.parquet", compression="snappy", index=False)

['@uob.parquet', '@AFFIN.parquet', '@smbcgroup.parquet', '@channelocbc.parquet', '@CIMBGroupHoldingsBhd.parquet', '@PublicBankGroup.parquet', '@MizuhoAmericas.parquet', '@HSBC_MY.parquet', '@AmBankTV.parquet', '@rhbgroup.parquet', '@BankofChinaManila.parquet', '@MUFGBankChannel.parquet', '@AllianceBankMY.parquet', '@maybankvideos.parquet', '@BankofAmerica.parquet', '.ipynb_checkpoints', '@standardchartered.parquet', '@GXBank.parquet', '@bangkokbankchannel.parquet', '@hongleongbankmy.parquet', '@labanquedunmondequichange.parquet', '@Citi.parquet', '@jpmorgan.parquet', '@DeutscheBank.parquet', '@MyBoostApp.parquet']
Metadata:



,Youtube_Video_IDs,Youtube_Handle,video_length,views,date,title,likes,description,MM_timestamp,is_commented
0,Y0b3KoxDmc8,@uob,2:05,61177,2023-10-02,Be Cyber Savvy: The Interceptor,29,Saw a great deal online? A tempting ad may jus...,2025-11-02 19:33:31,False
0,Y0b3KoxDmc8,@uob,2:05,61177,2023-10-02,Be Cyber Savvy: The Interceptor,29,Saw a great deal online? A tempting ad may jus...,2025-11-02 19:33:31,False


Comments:



,Youtube_Video_IDs


Metadata:



...

,Youtube_Video_IDs,poster_OP,comment_age,full_comment,like_count
0,-02nefEBp5s,@cybi2684,6 days ago (edited),"Haha... Stablecoin, ja genau. Der EURO wird se...",0
1,yPBxR0MnKXQ,@tormagnuslarsen9870,12 days ago,WTM World Trade Marketing WTM ™,1
2,a7pwVINtKV8,@俊霖陳-k7x,12 days ago (edited),我愛我是德國銀行男女能源之星你人真好 ❤,1
3,a7pwVINtKV8,@MayurMisra,11 days ago,"""Introducing dbX - The Future of Corpoarte Tre...",0
4,NgdCIqcCdRc,@Belisar505,12 days ago,Anfang der 80er Jahre war die letzte zyklische...,0
84,3Y_apyNmUs0,@michi2525,1 year ago,I am lucky tow hav you 😊,1
85,zUyVd36XiFs,@stefanbarnikow7069,1 year ago,Schön das es diesen Podcast wieder gibt. Das W...,1
86,3_K1RvaMUHI,@TheGabrielbraga,1 year ago,Mclovin i love tu gato ❤,0
87,3_K1RvaMUHI,@trggrt225,1 year ago,My ASSETS I WANT TO START UP MY INDEPENDENT PR...,0
88,3_K1RvaMUHI,@trggrt225,1 year ago,You need to know like I not ask any minister t...,0


Metadata:



,Youtube_Video_IDs,Youtube_Handle,video_length,views,date,title,likes,description,MM_timestamp,is_commented
0,JjMcghmK9ng,@MyBoostApp,2025-11-02 00:12:00,3740,2025-10-20,Why No One Tell Me I Can Get RM1.95/L for RON95?,1,"Many people ask, how is it possible to get RM1...",2025-10-30 03:53:32,False
1,uiEe-l1LfJM,@MyBoostApp,2025-11-02 00:15:00,15093,2025-10-15,Harga RON95 jadi RM1.95 jer seliter?,1,Harga RON95 jadi RM1.95 jer seliter?\n\nMemang...,2025-10-30 03:53:49,False
2,Fq7W7ytCMz0&pp=0gcJCQYKAYcqIYzv,@MyBoostApp,2025-11-02 00:20:00,52349,2025-10-10,Boost Epic Unstoppable Birthday,0,"Spend a minimum of RM30 using Boost, Boost Ban...",2025-10-30 03:54:06,False
3,gMbiTUi0dgw,@MyBoostApp,2025-11-02 00:14:00,81193,2025-10-09,Win Gold Dinar Boost Epic Unstoppable,2,🎉 More epic surprises are dropping for Boost’s...,2025-10-30 03:54:23,False
4,BwfBq2jzAyA,@MyBoostApp,2025-11-02 00:17:00,94484,2025-10-08,Boost Epic Unstoppable Birthday 2025,2,It’s an epic spending spree with Boost & Boost...,2025-10-30 03:54:40,False
225,C2Ikb1u3pSU,@MyBoostApp,2025-11-02 01:00:00,737,2021-02-26,#ShopMemangOnz with up to RM150 Boost Cashback...,11,2021 Shop Malaysia Online (SMO) Campaign is ba...,2025-11-01 01:07:20,None
226,tIyV7Hf6RVY,@MyBoostApp,2025-11-02 01:00:00,74858,2021-08-06,8.8 Online Boost Day 2021 | #ShopMemangOnz wit...,6,Boost eWallet is bringing you the biggest Onli...,2025-11-01 01:07:43,None
227,HjR8Am-CNhc,@MyBoostApp,2025-11-02 00:40:00,1627,2021-02-26,Boost Business – Accept Boost Payment via your...,10,It’s important to stay safe and do your part t...,2025-11-01 01:08:08,None
228,GaJ49C_iKYw,@MyBoostApp,2025-11-02 02:03:00,1038,2021-07-28,Boost Business - How to Setup Your Very Own Wh...,6,Are you facing the challenge of having to open...,2025-11-01 01:08:45,None
229,61q1a3TCO-4,@MyBoostApp,2025-11-02 03:47:00,1206717,2021-05-10,Iklan Raya Boost 2021: Hadiah Buat Mama,251,Pandemik ini bukan mudah bagi semua. Tetapi de...,2025-11-01 01:09:09,None


Comments:



,Youtube_Video_IDs,poster_OP,comment_age,full_comment,like_count
0,oiVhAT47iDs,@topethidayat1620,2 months ago,Merdeka kewangan! Merdeka kewangan! 🎉 🎉 🎉,3
1,oiVhAT47iDs,@eappieflag,2 months ago,bayar bill guna boost -bill tidak berjaya dib...,2
2,oiVhAT47iDs,@mz6228,2 months ago,Selamat Hari Kebangsaan Malaysia 🇲🇾 Merdeka!! ...,0
3,oiVhAT47iDs,@muhammadhazim1581,2 months ago,vfx need a raise,1
4,oiVhAT47iDs,@mdshaidik9275,1 month ago,What's with that fever patch? Hinting somethin...,0
87,W4DnaPnso8s&pp=0gcJCQYKAYcqIYzv,@masterdarkfender8272,1 year ago,My boost is broken. It's still loading even th...,0
88,W4DnaPnso8s&pp=0gcJCQYKAYcqIYzv,@OfficialP29,1 year ago,Nama ni,0
89,W4DnaPnso8s&pp=0gcJCQYKAYcqIYzv,@YukjingLim,1 year ago,Your boost credit team banyak masalah sudah sa...,0
90,W4DnaPnso8s&pp=0gcJCQYKAYcqIYzv,@chongweanne7818,1 year ago,"hello, your credit team banyak masalah dan tak...",0
91,W4DnaPnso8s&pp=0gcJCQYKAYcqIYzv,@wanchareesrinual7613,10 months ago,I open already 1 account and try to put money ...,0


## Delete statements for pickle

In [156]:
root_dirs = [
    "./Data/Database/",
    "./Data/Lists/MM_temp/",
    "./Data/Lists/COMMENT_temp/",
    "./Data/MM_database/",
    "./Data/COMMENT_database/"
]
delete_template = """find {placeholder} -name "*.pkl" -type f -delete"""

for e_directory in root_dirs[1:]:
    del_command = delete_template.format(placeholder=e_directory)
    print(del_command)

find ./Data/Lists/MM_temp/ -name "*.pkl" -type f -delete
find ./Data/Lists/COMMENT_temp/ -name "*.pkl" -type f -delete
find ./Data/MM_database/ -name "*.pkl" -type f -delete
find ./Data/COMMENT_database/ -name "*.pkl" -type f -delete


## Data Cleaning video_length/duration ?

In [177]:
COLLATE_MM.groupby(pd.Grouper(key='MM_timestamp', axis=0, freq='D'))['MM_timestamp'].count()

MM_timestamp
2025-10-30    2400
2025-10-31    2520
2025-11-01    7056
Freq: D, Name: MM_timestamp, dtype: int64

In [ ]:
# Convert to nullable integer and boolean dtypes
df['col_a'] = df['col_a'].astype('Int64')
df['col_b'] = df['col_b'].astype('BooleanDtype')

In [768]:
nun = None
if nun:
    print('A')
else:
    print('Yes')

Yes


In [767]:
print(MM_df.dtypes)
print()
print(comment_df.dtypes)
display(MM_df.head())

views                    int64
date            datetime64[ns]
title                   object
likes                    int64
description             object
MM_timestamp    datetime64[ns]
is_commented            object
dtype: object

Series([], dtype: object)


,views,date,title,likes,description,MM_timestamp,is_commented
0,276,2024-11-22,SMBC Careers | Oge Udensi on securing the future of finance,4,"Meet Oge Udensi, the EMEA lead for Cyber Governance, Risk & Compliance. Learn about her journey and commitment to empowering women in technology. Discover how SMBC is prioritising innovation and transformation.\n\nSMBC is a global banking leader operating across three core divisions: Corporate & Investment Banking, Structured Finance and Global Markets. We empower our employees to think big on an international scale, while fostering a collaborative spirit that keeps everyone connected. We're constantly searching for curious and ambitious minds to join our team and drive our ongoing success. Discover what a future at SMBC looks like at:\nhttps://www.smbcgroup.com/emea/careers",2025-10-28 17:06:53,False


In [283]:
for el in comments:
    # Get any emojis
    emojies = el.find_elements(By.XPATH, ".//img[@alt]")
    for e_emoji in emojies:
        pass
        print(e_emoji.get_attribute('alt'))
    # Get the replies 
    replies = el.find_elements(*reply_comments_path)
    # print(replies.text)
    small_comment_dict = {}
    # Process the text
    li = el.text.split('\n')
    lit = list(filter(lambda x : not re.search(pattern=r"^Pinned", string=x), li))
    print(lit[-2])
    poster, comment_age, *comment_text_content, like_count, reply = lit
    full_comment =' '.join(comment_text_content)
    small_comment_dict.update({'poster_OP' : poster,
                               'comment_age' : comment_age,
                               'full_comment' : full_comment,
                               'like_count' : like_count,
                               'MM_timestamp' : datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
                              })
    COMMENT_DICT.append(small_comment_dict)
    print("-----")


🙏
❤
17
-----

😍
❤
4
-----

🎉
❤
2
-----

❤
🎉
1
-----

Syabas for the tot
-----

👍
👌
🥰
🥰
🥰
1
-----

😊
1 hour ago


ValueError: not enough values to unpack (expected at least 4, got 3)

In [298]:
for el in comments:
    small_comment_dict = {}

    # Get the HTML of the comment text section
    try:
        content_element = el.find_elements(By.XPATH, ".//yt-attributed-string[@id='content-text']")
        html_content = content_element.get_attribute("innerHTML")
    except Exception:
        html_content = ""
    print(html_content)

    # Replace emoji <img> tags with their alt text (like 😍🔥)
    # Example: <img ... alt="🔥"> → "🔥"
    html_content = re.sub(r'<img[^>]+alt="([^"]+)"[^>]*>', r'\1', html_content)

    # Remove any HTML tags (like <a>, <b>, etc.)
    full_comment = re.sub(r'<[^>]+>', '', html_content).strip()

    # Now continue as before
    li = el.text.split('\n')
    lit = list(filter(lambda x : not re.search(pattern=r"^Pinned", string=x), li))

    try:
        poster, comment_age, *comment_text_content, like_count, reply = lit
    except ValueError:
        # fallback in case structure is inconsistent
        continue

    small_comment_dict.update({
        'poster_OP': poster,
        'comment_age': comment_age,
        'full_comment': full_comment,  # ✅ includes emojis
        'like_count': like_count,
        'MM_timestamp': datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
    })

    COMMENT_DICT.append(small_comment_dict)
    print(full_comment)
    print("-----")




-----


-----


-----


-----


-----


-----



-----


-----


-----


## Appendix

In [ ]:
 ## --------- SCROLLING -------------------------
    max_num_of_scrolls = None
    num_of_scrolls = 0
    pause_time=2
    max_attempts=3
    
    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    same_height_attempts = 0

    
    print("🔄 Starting infinite scroll...")

    is_scrape_comments = True
    
    while same_height_attempts < max_attempts:
        if num_of_scrolls == max_num_of_scrolls:
            print(f"Max num of scrolls reached. Breaking out.")
            break
        # Scroll to the bottom
        # driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        
        # Wait for page to load new items
        time.sleep(pause_time)
        
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        
        # Wait for page to load new items
        time.sleep(pause_time)
        # (Optional) move browser window using pyautogui
        pyautogui.hotkey('ctrl', 'alt', 'left')
        # Wait for page to load new items
        time.sleep(pause_time)

        if num_of_scrolls == 0:
            print('First scroll, but not increment num_of_scrolls')
            
            # Wait for page to load new items
            time.sleep(pause_time)
            # (Optional) move browser window using pyautogui
            pyautogui.hotkey('ctrl', 'alt', 'left')
            # Wait for page to load new items
            time.sleep(pause_time)
            
            # First make a quick check that there are comments
            is_comments_path = (By.XPATH, "//ytd-comments-header-renderer[contains(@class, 'style-scope')]")
            is_comments = driver.find_elements(*is_comments_path)
            print("-----------COMMENTS-----------------------")
            for i, el in enumerate(is_comments):
                rect = el.rect
                displayed = el.is_displayed()
                print(f"[{i}] Tag: {el.tag_name}, Visible: {displayed}, Rect: {rect}")
                if displayed:
                    clickable_element = el
            hope_text_list = clickable_element.find_elements(By.XPATH, ".//yt-formatted-string[contains(@class, 'count-text')]")
            if hope_text_list:
                for e_text in hope_text_list:
                    print(e_text.text)
                    match = re.search(pattern=r'^(\d+)\b\s+\bComments', string=e_text.text)
                    # print(match.group(0))
                    if match :
                        if match.group(1) == '0':
                            print('Uncommented video')
                            MM_collector.update({'is_commented' : False})
                            is_scrape_comments = False
                            break
                        else:
                            MM_collector.update({'is_commented' : True})
                            pass # We can scrape_comments
            else: # This means comments are diasbled
                print('Disabled_comment_video')
                MM_collector.update({'is_commented' : None})
                is_scrape_comments = False

        if new_height == last_height:
            same_height_attempts += 1
            print(f"⚠️ No new content... ({same_height_attempts}/{max_attempts})")
        else:
            same_height_attempts = 0
            last_height = new_height
            print("✅ New content loaded.")
            num_of_scrolls += 1

In [325]:
url = r"https://www.youtube.com/watch?v=s9pNJYzRvxs&list=RDs9pNJYzRvxs&start_radio=1"

service = Service("/usr/local/bin/chromedriver")
driver = webdriver.Chrome(service=service)
driver.get(url)

# (Optional) move browser window using pyautogui
pyautogui.hotkey('ctrl', 'alt', 'left')

wait = WebDriverWait(driver, 30)

more_info = (By.XPATH, '//tp-yt-paper-button[contains(@id, "expand")]')
class1 = (By.CLASS_NAME, "button style-scope ytd-text-inline-expander")
xpath_2 = (By.XPATH, "//yt-formatted-string[contains(@id, 'info')]")
comments_path = (By.XPATH, "//ytd-comment-view-model[contains(@id, 'comment')]")
reply_comments_path = (By.XPATH, ".//div[contains(@id, 'collapsed-threads')]")
like_button_path = (By.XPATH, "//button[contains(@aria-label, 'like')]")
replies_button = (By.XPATH, "//button[contains(@aria-label, 'replies')]")


MM_collector = {}

# Next we have to wait for the page to load
WebDriverWait(driver, 15).until(
    EC.presence_of_element_located(xpath_2)
)
# Still a bit of lag though
time.sleep(3)


## ---------------VIEWS_AND_DATE----------------------------
# Now that we know our element exists, we can click it -THIS DOES NOT WORK
# Check which elements are actually interactable
elements = driver.find_elements(*more_info) # use plural
for i, el in enumerate(elements):
    rect = el.rect
    displayed = el.is_displayed()
    print(f"[{i}] Tag: {el.tag_name}, Visible: {displayed}, Rect: {rect}")
    if displayed:
        clickable_element = el

# From above we know the last element is the interactable element we can click to expand : driver.find_elements(*more_info)[-1].click()
# Check if we found any interactable element
if clickable_element:
    # Scroll into view before clicking (good practice)
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", clickable_element)
    clickable_element.click()
    print("✅ Clicked visible 'expand' element.")

# Next we have to explicitly wait for the new content to render
time.sleep(3)
# Next we have to wait for the page to load
WebDriverWait(driver, 15).until(
    EC.presence_of_element_located(xpath_2)
)

# Collect all loaded elements
views_and_date = driver.find_elements(*xpath_2)[-1].find_elements(By.TAG_NAME, 'span')

for idx, e_child_ele in enumerate(views_and_date, start=1):
    some_info = e_child_ele.text
    if idx == 1:
        MM_collector.update({'views' : some_info})
    elif idx == 2:
        MM_collector.update({'date' : some_info})


## ---------------LIKE-BUTTON ----------------------------
# First grab all buttons with aria-label attribute
like_button = driver.find_elements(*like_button_path)
for i, el in enumerate(like_button):
    rect = el.rect
    displayed = el.is_displayed()
    print(f"[{i}] Tag: {el.tag_name}, Visible: {displayed}, Rect: {rect}")
    if displayed:
        clickable_element = el
        break

# Then filter with regex
pattern = re.compile(r"^like.*", re.IGNORECASE)
like_info_match = pattern.search(string=clickable_element.get_attribute("aria-label"))
if like_info_match:
    like_string = clickable_element.get_attribute('aria-label')
    print(like_string)
    thousand_comma_separation_pattern = re.compile(r"\d{1,3}(?:,\d{3})*(?:\.\d*)?", re.IGNORECASE)
    number = thousand_comma_separation_pattern.search(string=like_string)
    print(f"Here is the like-count : {correct_number(number.group(0))}")
    MM_collector.update({'likes' : correct_number(number.group(0))})

##---------------DESCRIPTION ----------------------------
# Inspect which elements are interactable
description = driver.find_elements(By.XPATH, "//div[contains(@id, 'expanded')]") # First path
for i, el in enumerate(description):
    rect = el.rect
    displayed = el.is_displayed()
    print(f"[{i}] Tag: {el.tag_name}, Visible: {displayed}, Rect: {rect}")
    if displayed:
        clickable_element = el

    
# Use relative ".//" to search within nested elements/descendants 'under' descripton[-1]
many_spans = clickable_element.find_element(By.XPATH, ".//yt-attributed-string[contains(@class, 'ytd-text-inline-expander')]")\
                              .find_elements(By.XPATH, ".//span[contains(@class, 'yt-core-attributed-string--link-inherit-color')]") # third path

description_list = []
count = 0
for e_span in many_spans:
    clean_str = e_span.text.strip()
    description_list.append(clean_str)
    # try:
    #     if re.search(pattern=r':\s*$', string=clean_str):
    #         description_string.append
    #     elif re.search(pattern=r'[\.\s]\s*$', string=clean_str):
    #         description_string +=  e_span.text + '\n'
    #     else:
    #         description_string += e_span.text
    # except NoSuchElementException:
    #     external_webisite_link = e_span.find_element(By.XPATH, "//a[@href]")
    #     description_string += e_span.text

    description_string = '\n'.join(description_list)

    count += 1

print(f"This is the description:\n{description_string}")
MM_collector.update({'description' : description_string})

max_num_of_scrolls = 2
num_of_scrolls = 0
pause_time=2
max_attempts=2

last_height = driver.execute_script("return document.documentElement.scrollHeight")
same_height_attempts = 0
print("🔄 Starting infinite scroll...")

while same_height_attempts < max_attempts:
    if num_of_scrolls == max_num_of_scrolls:
        print(f"Max num of scrolls reached. Breaking out.")
        break
    # Scroll to the bottom
    # driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    
    # Wait for page to load new items
    time.sleep(pause_time)
    
    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    
    # Wait for page to load new items
    time.sleep(pause_time)
    
    if new_height == last_height:
        same_height_attempts += 1
        print(f"⚠️ No new content... ({same_height_attempts}/{max_attempts})")
    else:
        same_height_attempts = 0
        last_height = new_height
        print("✅ New content loaded.")
        num_of_scrolls += 1

    #     # Grab all buttons with aria-label
    #     buttons = driver.find_elements(By.XPATH, "//button[@aria-label]")
        
    #     pattern = re.compile(r"repl(y|ies)", re.IGNORECASE)
        
    #     for btn in buttons:
    #         label = btn.get_attribute("aria-label") or ""
    #         if pattern.search(label):
    #             try:
    #                 # Scroll into view first
    #                 driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", btn)
    #                 time.sleep(0.5)
    #                 # Fallback: JS click
    #                 driver.execute_script("arguments[0].click();", btn)
    #                 print(f"✅ JS clicked button: {label}")
                    
                    
        
    #             except Exception as e:
    #                 # Try normal click
    #                 btn.click()
    #                 print(f"Clicked button: {label}")
    #                 print(f"⚠️ Normal click failed on '{label}': {e}")
                    
    #             num_of_scrolls += 1

        



        # buttons = driver.find_elements(By.XPATH, "//button[@aria-label]")
        # for i, el in enumerate(buttons):
        #     print('Reply buttons')
        #     rect = el.rect
        #     displayed = el.is_displayed()
        #     print(f"[{i}] Tag: {el.tag_name}, Visible: {displayed}, Rect: {rect}")
        #     if displayed:
        #         clickable_element = el
        #         # Click the clickable reply button 
        #         if clickable_element:
        #             pattern = re.compile(r"reply|replies", re.IGNORECASE)
        #             match = pattern.search(string=clickable_element.get_attribute("aria-label"))
        #             if match:
        #                 driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", clickable_element)
        #                 clickable_element.click()
        #                 print("✅ Clicked visible 'reply' element.")
                        
        #             time.sleep(2)
        #     else:
        #         continue
            
                
            # reply_string = clickable_element.get_attribute('aria-label')
            # # Then filter with regex
            # pattern = re.compile(r"repl(y|ies)", re.IGNORECASE)
        

print("🛑 Finished scrolling.")
print("🔍 Expanding first-level replies...")

reply_pattern = re.compile(r"\b\d+\s*repl(y|ies)\b", re.IGNORECASE)
max_scrolls = 6
expanded_labels = set()
total_clicked = 0

expanded_threads = set()

for scroll_round in range(max_scrolls):
    print(f"🌀 Pass {scroll_round+1}: scanning for reply buttons...")

    buttons = driver.find_elements(By.XPATH, "//button[@aria-label]")
    click_candidates = []

    for btn in buttons:
        label = btn.get_attribute("aria-label") or ""
        if reply_pattern.search(label):
            # Identify unique ancestor comment container
            ancestor = btn.find_element(By.XPATH, "./ancestor::ytd-comment-thread-renderer")
            comment_id = ancestor.get_attribute("id") or str(ancestor.id)

            if comment_id not in expanded_threads:
                click_candidates.append((btn, comment_id))

    if not click_candidates:
        print("⚠️ No new reply buttons found.")
        break
    else:
        print('CLICK CANDIDATES')
        print(click_candidates)

    for btn, comment_id in click_candidates:
        try:
            if comment_id not in expanded_threads:
                label = btn.get_attribute("aria-label")
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", btn)
                time.sleep(0.7)
                driver.execute_script("arguments[0].click();", btn)
                expanded_threads.add(comment_id)
                total_clicked += 1
                print(f"✅ Expanded replies: {label}")
                time.sleep(2)
            else:
                continue
        except Exception as e:
            print(f"⚠️ Failed to click '{label}': {e}")
            continue

    # Scroll a bit further to load more comment threads
    driver.execute_script("window.scrollBy(0, 800);")
    time.sleep(2)

print(f"🎯 Finished expanding {total_clicked} first-level reply threads.")

# --- Step 2: Collect comments + first-level replies ---
print("🧩 Extracting comments and replies...")
COMMENT_DICT = []


comments = driver.find_elements(By.XPATH, "//ytd-comment-view-model[contains(@id, 'comment')]")
for i, el in enumerate(comments):
    print("COMMENT DIV")
    rect = el.rect
    displayed = el.is_displayed()
    print(f"[{i}] Tag: {el.tag_name}, Visible: {displayed}, Rect: {rect}")


for idx, el in enumerate(comments):
    try:
        li = el.text.split('\n')
        lit = list(filter(lambda x : not re.search(pattern=r"^Pinned", string=x), li))
        # Handle variable-length lists gracefully
        if len(lit) < 4:
            continue
        poster, comment_age, *comment_content = lit[:-2]
        like_count, reply_text = lit[-2:]
        full_comment = ' '.join(comment_content).strip()

        small_comment_dict = {
            'poster_OP': poster,
            'comment_age': comment_age,
            'full_comment': full_comment,
            'like_count': like_count,
            'replies': [],
            'MM_timestamp': datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
        }

        # Step 3: Find replies under this comment (relative path)
        # reply_divs = el.find_elements(By.XPATH, ".//div[@id='expander-contents']")
        # reply_divs = el.find_elements(By.XPATH, ".//ytd-comment-view-model[contains(@class, 'style-scope yrd-comment-replies-renderer')]")
        # reply_divs = el.find_elements(By.XPATH, ".//span[contains(@class, 'yt-core-attributed-string yt-core-attributed-string--white-space-pre-wrap')]")
        # # Run Diagnostic failes to produce reply_divs
        # for i, el in enumerate(reply_divs):
        #     print("REPLY DIVS")
        #     rect = el.rect
        #     displayed = el.is_displayed()
        #     print(f"[{i}] Tag: {el.tag_name}, Visible: {displayed}, Rect: {rect}")
        #     if displayed:
        #         clickable_element = el
        
        #     hope_text = clickable_element.find_element(By.XPATH, ".//yt-attributed-string[contains(@id, 'content-text')]") 
        #     try:
        #         reply_lines = hope_text.text #split('\n')
        #         if len(reply_lines) >= 4:
        #             reply_poster = reply_lines[0]
        #             reply_age = reply_lines[1]
        #             reply_content = ' '.join(reply_lines[2:-2]).strip()
        #             reply_likes = reply_lines[-2]
        #             small_comment_dict['replies'].append({
        #                 'poster_reply': reply_poster,
        #                 'comment_age': reply_age,
        #                 'full_comment': reply_content,
        #                 'like_count': reply_likes
        #             })
        #     except Exception as e:
        #         print(f"⚠️ Failed to parse reply: {e}")

        # Find reply containers within the comment
        reply_divs = el.find_elements(
            By.XPATH,
            ".//ytd-comment-replies-renderer//yt-attributed-string[@id='content-text']"
        )
        
        print("🔍 Found", len(reply_divs), "reply_divs")
        for i, reply_el in enumerate(reply_divs):
            rect = reply_el.rect
            displayed = reply_el.is_displayed()
            print(f"[{i}] Tag: {reply_el.tag_name}, Visible: {displayed}, Rect: {rect}")
        
            if not displayed:
                continue  # skip hidden ones
        
            # Extract text and emojis in DOM order
            children = reply_el.find_elements(By.XPATH, "./*")
            reconstructed = []
            for node in children:
                tag = node.tag_name.lower()
                if tag == "span":
                    reconstructed.append(node.text.strip())
                elif tag == "img":
                    reconstructed.append(node.get_attribute("alt") or "")
            text = "".join(reconstructed)
        
            print(f"💬 Reply #{i}: {text}")

        COMMENT_DICT.append(small_comment_dict)
        print(f"🗨️ [{idx}] Collected comment by {poster} with {len(small_comment_dict['replies'])} replies.")
    except Exception as e:
        print(f"⚠️ Failed to parse comment #{idx}: {e}")

print(f"\n✅ Total top-level comments collected: {len(COMMENT_DICT)}")


## Add function to click replies
# First grab all buttons with aria-label attribute


    
#     target_buttons = [b for b in buttons if pattern.search(b.get_attribute("aria-label"))]
# # # Click them
# for btn in target_buttons:
#     driver.find_element(btn).click()
#     time.sleep(2)
#     print(f'Clicked the reply button')

# for btn in buttons:
#     label = btn.get_attribute("aria-label") or ""
#     if pattern.search(label):
#         try:
#             # Scroll into view first
#             driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", btn)
#             time.sleep(0.5)
            
#             # # Try normal click
#             # btn.click()
#             # print(f"Clicked button: {label}")
#             driver.execute_script("arguments[0].click();", btn)
#             print(f"✅ JS clicked button: {label}")

#         except Exception as e:
#             print(e)
#             # print(f"⚠️ Normal click failed on '{label}': {e}")
#             # Fallback: JS click
            


# Collect all loaded comments


# COMMENT_DICT = []
# comments = driver.find_elements(*comments_path)

# for el in comments:
#     # Get the replies 
#     replies = el.find_elements(*reply_comments_path)
#     # print(replies.text)
#     small_comment_dict = {}
#     # Process the text
#     li = el.text.split('\n')
#     lit = list(filter(lambda x : not re.search(pattern=r"^Pinned", string=x), li))
#     # print(lit)
#     poster, comment_age, *comment_content, like_count, reply = lit
#     full_comment =' '.join(comment_content)
#     small_comment_dict.update({'poster_OP' : poster,
#                                'comment_age' : comment_age,
#                                'full_comment' : full_comment,
#                                'like_count' : like_count,
#                                'MM_timestamp' : datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
#                               })
#     COMMENT_DICT.append(small_comment_dict)
#     print("-----")



    # my_string = el.find_element(By.XPATH, '//yt-attributed-string[contains(@id, "content-text")]')
    # attribute_value = my_string.find_element(By.XPATH, '//span[contains(@role, "text")]')
    # print(attribute_value.text)


    # return views_and_date, description_string, driver  # return driver if you want to extract text/attributes later
        

[0] Tag: tp-yt-paper-button, Visible: False, Rect: {'height': 0, 'width': 0, 'x': 0, 'y': 0}
[1] Tag: tp-yt-paper-button, Visible: False, Rect: {'height': 0, 'width': 0, 'x': 0, 'y': 0}
[2] Tag: tp-yt-paper-button, Visible: False, Rect: {'height': 20, 'width': 44, 'x': 188.1796875, 'y': 1163.75}
[3] Tag: tp-yt-paper-button, Visible: True, Rect: {'height': 20, 'width': 44, 'x': 188, 'y': 1163.75}
✅ Clicked visible 'expand' element.
[0] Tag: button, Visible: False, Rect: {'height': 0, 'width': 0, 'x': 0, 'y': 768.5}
[1] Tag: button, Visible: False, Rect: {'height': 0, 'width': 0, 'x': 0, 'y': 768.5}
[2] Tag: button, Visible: False, Rect: {'height': 0, 'width': 0, 'x': 0, 'y': 768.5}
[3] Tag: button, Visible: False, Rect: {'height': 0, 'width': 0, 'x': 0, 'y': 768.5}
[4] Tag: button, Visible: True, Rect: {'height': 36, 'width': 80, 'x': 329.765625, 'y': 1040.5}
like this video along with 353 other people
Here is the like-count : 353
[0] Tag: div, Visible: False, Rect: {'height': 0, 'width

In [326]:
COMMENT_DICT

[{'poster_OP': '@kaminimanikam', 'comment_age': '9 days ago (edited)', 'full_comment': 'Thank you Ambank for celebrating Bharatanatyam — and for choosing me to embody its spirit in this meaningful tapestry of light, rhythm, and story. I enjoyed the filming of this advertisement. Happy Deepavali everyone! -Kamini Manikam-', 'like_count': '17', 'replies': [], 'MM_timestamp': '10/27/2025, 01:43:32'}, {'poster_OP': '@RoseEsrosy', 'comment_age': '11 days ago', 'full_comment': 'So beautiful   well done Ambank', 'like_count': '4', 'replies': [], 'MM_timestamp': '10/27/2025, 01:43:32'}, {'poster_OP': '@keshavbaralofficial', 'comment_age': '11 days ago', 'full_comment': 'Wow  love from Nepal 🇳🇵️', 'like_count': '3', 'replies': [], 'MM_timestamp': '10/27/2025, 01:43:32'}, {'poster_OP': '@lalithaolaal6879', 'comment_age': '9 days ago', 'full_comment': 'Vauuuuuu,beautiful  Am bank', 'like_count': '1', 'replies': [], 'MM_timestamp': '10/27/2025, 01:43:32'}, {'poster_OP': '@premagopalan2276', 'comme

In [246]:
for el in comments:
    # Get the replies 
    replies = el.find_elements(*reply_comments_path)
    print(replies)
    small_comment_dict = {}
    # Process the text
    li = el.text.split('\n')
    lit = list(filter(lambda x : not re.search(pattern=r"^Pinned", string=x), li))
    # print(lit)
    poster, comment_age, *comment_content, like_count, reply = lit
    full_comment =' '.join(comment_content)
    small_comment_dict.update({'poster_OP' : poster,
                               'comment_age' : comment_age,
                               'full_comment' : full_comment,
                               'like_count' : like_count,
                               'MM_timestamp' : datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
                              })
    COMMENT_DICT.append(small_comment_dict)
    print("-----")

[]
-----
[]
-----
[]
-----
[]
-----
[]
-----
[]
-----
[]
-----
[]
-----
[]
-----


In [230]:
clickable_element.get_attribute('aria-label')

In [ ]:
from 

In [117]:
my_string = "This is a sentence"
word_list = my_string.split()
print(word_list)

# word_list_2 = comments[0].text.split('\n')
# print(word_list_2)

lengths = []
COMMENT_DICT = []

for el in comments:
    small_comment_dict = {}
    li = el.text.split('\n')
    lit = list(filter(lambda x : not re.search(pattern=r"^Pinned", string=x), li))
    # if len(lit) > 5:
    print(lit)
    poster, comment_age, *comment_content, like_count, reply = lit
    full_comment =' '.join(comment_content)
    small_comment_dict.update({'poster' : poster,
                       'comment-age' : comment_age,
                       'full_comment' : full_comment,
                       'like_count' : like_count,
                       'MM_timestamp' : datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
                      })
    COMMENT_DICT.append(small_comment_dict)
    
        
    lengths.append(len(li))

['This', 'is', 'a', 'sentence']
['@PrisonPlanetLive', '4 months ago', 'Go to https://ground.news/pjw or download the app and use my code P J W to access all perspectives and see through biased media. Subscribe through my link for 40% off unlimited access.', '194', 'Reply']
['@therealjoemullin', '4 months ago', 'In South africa, the blacks ransacked an entire mall and left one shop utterly pristine and untouched. A book store.', '4K', 'Reply']
['@mrthewubbie', '4 months ago', '"During the Soviet period, you could take someone that believes in the party, place them in the middle of a gulag, show them the truth, and they still would refuse to believe you". Yuri Besmenov', '3.1K', 'Reply']
['@AryBoy1998', '4 months ago', '“I can’t speak English”', '', 'Right so you’re living in England but can’t speak English? Well done.', '1.4K', 'Reply']
['@bluestone476', '4 months ago', "It doesn't suck if they get what they voted for.... what really sucks is that WE get what THEY voted for.", '728', 'R

In [118]:
COMMENT_DICT

[{'poster': '@PrisonPlanetLive',
  'comment-age': '4 months ago',
  'full_comment': 'Go to https://ground.news/pjw or download the app and use my code P J W to access all perspectives and see through biased media. Subscribe through my link for 40% off unlimited access.',
  'like_count': '194',
  'MM_timestamp': '10/24/2025, 17:12:50'},
 {'poster': '@therealjoemullin',
  'comment-age': '4 months ago',
  'full_comment': 'In South africa, the blacks ransacked an entire mall and left one shop utterly pristine and untouched. A book store.',
  'like_count': '4K',
  'MM_timestamp': '10/24/2025, 17:12:50'},
 {'poster': '@mrthewubbie',
  'comment-age': '4 months ago',
  'full_comment': '"During the Soviet period, you could take someone that believes in the party, place them in the middle of a gulag, show them the truth, and they still would refuse to believe you". Yuri Besmenov',
  'like_count': '3.1K',
  'MM_timestamp': '10/24/2025, 17:12:50'},
 {'poster': '@AryBoy1998',
  'comment-age': '4 mo

In [155]:
example = ['Pinned by @PrisonPlanetLive', '@PrisonPlanetLive', '1 year ago', 'Go to https://sheathunderwear.com/PRISONPLANET or use the code PRISONPLANET to get 20% off your Sheath underwear today.', '247', 'Reply']

ex = list(filter(lambda x : not re.search(pattern=r'^@', string=x), example))
poster, comment_age, *comment_content, lie_count, reply = lit


['Imagine a country without Kimmel.']

In [85]:
from bs4 import BeautifulSoup
my_html="""<button class="yt-spec-button-shape-next yt-spec-button-shape-next--text yt-spec-button-shape-next--call-to-action yt-spec-button-shape-next--size-m yt-spec-button-shape-next--icon-leading yt-spec-button-shape-next--align-by-text yt-spec-button-shape-next--enable-backdrop-filter-experiment" title="" aria-label="32 replies"><div aria-hidden="true" class="yt-spec-button-shape-next__icon"><span class="ytIconWrapperHost" style="width: 24px; height: 24px;"><span class="yt-icon-shape ytSpecIconShapeHost"><div style="width: 100%; height: 100%; display: block; fill: currentcolor;"><svg xmlns="http://www.w3.org/2000/svg" height="24" viewBox="0 0 24 24" width="24" focusable="false" aria-hidden="true" style="pointer-events: none; display: inherit; width: 100%; height: 100%;"><path d="m18 9.28-6.35 6.35-6.37-6.35.72-.71 5.64 5.65 5.65-5.65z"></path></svg></div></span></span></div><div class="yt-spec-button-shape-next__button-text-content"><span class="yt-core-attributed-string yt-core-attributed-string--white-space-no-wrap" role="text">32 replies</span></div><yt-touch-feedback-shape aria-hidden="true" class="yt-spec-touch-feedback-shape yt-spec-touch-feedback-shape--touch-response"><div class="yt-spec-touch-feedback-shape__stroke"></div><div class="yt-spec-touch-feedback-shape__fill"></div></yt-touch-feedback-shape></button>"""
soup = BeautifulSoup(my_html, 'html.parser')

# Get the prettified HTML
prettified_html = soup.prettify()
print(prettified_html)

<button aria-label="32 replies" class="yt-spec-button-shape-next yt-spec-button-shape-next--text yt-spec-button-shape-next--call-to-action yt-spec-button-shape-next--size-m yt-spec-button-shape-next--icon-leading yt-spec-button-shape-next--align-by-text yt-spec-button-shape-next--enable-backdrop-filter-experiment" title="">
 <div aria-hidden="true" class="yt-spec-button-shape-next__icon">
  <span class="ytIconWrapperHost" style="width: 24px; height: 24px;">
   <span class="yt-icon-shape ytSpecIconShapeHost">
    <div style="width: 100%; height: 100%; display: block; fill: currentcolor;">
     <svg aria-hidden="true" focusable="false" height="24" style="pointer-events: none; display: inherit; width: 100%; height: 100%;" viewbox="0 0 24 24" width="24" xmlns="http://www.w3.org/2000/svg">
      <path d="m18 9.28-6.35 6.35-6.37-6.35.72-.71 5.64 5.65 5.65-5.65z">
      </path>
     </svg>
    </div>
   </span>
  </span>
 </div>
 <div class="yt-spec-button-shape-next__button-text-content">
  

In [ ]:
<button class="yt-spec-button-shape-next yt-spec-button-shape-next--text yt-spec-button-shape-next--call-to-action yt-spec-button-shape-next--size-m yt-spec-button-shape-next--icon-leading yt-spec-button-shape-next--align-by-text yt-spec-button-shape-next--enable-backdrop-filter-experiment" title="" aria-label="32 replies"><div aria-hidden="true" class="yt-spec-button-shape-next__icon"><span class="ytIconWrapperHost" style="width: 24px; height: 24px;"><span class="yt-icon-shape ytSpecIconShapeHost"><div style="width: 100%; height: 100%; display: block; fill: currentcolor;"><svg xmlns="http://www.w3.org/2000/svg" fill="currentColor" height="24" viewBox="0 0 24 24" width="24" focusable="false" aria-hidden="true" style="pointer-events: none; display: inherit; width: 100%; height: 100%;"><path d="M18.707 8.793a1 1 0 00-1.414 0L12 14.086 6.707 8.793a1 1 0 10-1.414 1.414L12 16.914l6.707-6.707a1 1 0 000-1.414Z"></path></svg></div></span></span></div><div class="yt-spec-button-shape-next__button-text-content"><span class="yt-core-attributed-string yt-core-attributed-string--white-space-no-wrap" role="text">32 replies</span></div><yt-touch-feedback-shape aria-hidden="true" class="yt-spec-touch-feedback-shape yt-spec-touch-feedback-shape--touch-response"><div class="yt-spec-touch-feedback-shape__stroke"></div><div class="yt-spec-touch-feedback-shape__fill"></div></yt-touch-feedback-shape>
</button>

In [78]:
for el in comments:
    print(el.text)
    print('-------------------------------------')
    # my_string = el.find_elements(By.XPATH, '//yt-attributed-string[contains(@id, "content-text")]')[0]
    # print(my_string.text)
    # attribute_value = my_string.find_elements(By.XPATH, '//span[contains(@role, "text")]')[-1]
    # print(attribute_value.text)

Pinned by @PrisonPlanetLive
@PrisonPlanetLive
1 year ago
Go to https://sheathunderwear.com/PRISONPLANET or use the code PRISONPLANET to get 20% off your Sheath underwear today.
247
Reply
-------------------------------------
@Chicken_Wing91
1 year ago
Jimmy Kimmel has never been accused of being the smartest guy in the room to start with
1.2K
Reply
-------------------------------------
@Patrick_919
1 year ago (edited)
I feel sorry for the Japanese people. Having Jimmy Kimmel in their country must've been awful.
3.5K
Reply
-------------------------------------
@OneMeanArtist
1 year ago
Aside from Japan, I love how smoothbrain Kimmel assumes the entire US is just as filthy and third world as CA.
651
Reply
-------------------------------------
@cannonball666
1 year ago
Japan and Russia don't use criminals and drug addicts as voting blocks.
1.2K
Reply
-------------------------------------
@beckvitt2997
1 year ago
"Japan was the future, but it's stuck in the past."

Being stuck in the past 

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

url = "https://www.youtube.com/watch?v=kunuFjyvwoE"

service = Service("/usr/local/bin/chromedriver")
driver = webdriver.Chrome(service=service)
driver.get(url)

wait = WebDriverWait(driver, 15)
wait.until(EC.presence_of_element_located((By.TAG_NAME, "ytd-text-inline-expander")))

# Give YouTube time to hydrate the shadow DOM
time.sleep(3)

expand_button = driver.execute_script("""
    return document
      .querySelector('ytd-text-inline-expander')
      .shadowRoot
      .querySelector('tp-yt-paper-button#expand');
""")

if expand_button:
    expand_button.click()
    print("Clicked expand button successfully!")
else:
    print("Expand button not found inside shadow DOM.")


## Hello

In [42]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

url = "https://www.youtube.com/watch?v=kunuFjyvwoE"

service = Service("/usr/local/bin/chromedriver")
driver = webdriver.Chrome(service=service)
driver.get(url)

# (Optional) move browser window using pyautogui
pyautogui.hotkey('ctrl', 'alt', 'left')

wait = WebDriverWait(driver, 20)

# Wait until at least one expander exists
wait.until(EC.presence_of_element_located((By.TAG_NAME, "ytd-text-inline-expander")))
time.sleep(3)

expand_button = None
for attempt in range(5):
    try:
        expand_button = driver.execute_script("""
            const expander = document.querySelector('ytd-text-inline-expander');
            if (!expander) return null;
            const root = expander.shadowRoot;
            if (!root) return null;
            return root.querySelector('tp-yt-paper-button#expand');
        """)
        if expand_button:
            print(f"Found expand button on attempt {attempt+1}")
            expand_button.click()
            print("Clicked expand button successfully.")
            break
        else:
            print(f"Attempt {attempt+1}: expand button not found yet.")
            time.sleep(2)
    except Exception as e:
        print(f"Attempt {attempt+1}: JS error -> {e}")
        time.sleep(2)

if not expand_button:
    print("Expand button not found after retries.")


Attempt 1: expand button not found yet.
Attempt 2: expand button not found yet.
Attempt 3: expand button not found yet.
Attempt 4: expand button not found yet.
Attempt 5: expand button not found yet.
Expand button not found after retries.


In [ ]:
<tp-yt-paper-button id="expand" class="button style-scope ytd-text-inline-expander" style-target="host" role="button" tabindex="0" animated="" elevation="0" aria-disabled="false" style="left: 436px;"><!--css-build:shady--><!--css_build_scope:tp-yt-paper-button--><!--css_build_styles:video.youtube.src.web.polymer.shared.ui.styles.yt_base_styles.yt.base.styles.css.js,third_party.javascript.youtube_components.tp_yt_paper_button.tp.yt.paper.button.css.js-->...more
<tp-yt-paper-ripple class="style-scope tp-yt-paper-button"><!--css-build:shady--><!--css_build_scope:tp-yt-paper-ripple--><!--css_build_styles:video.youtube.src.web.polymer.shared.ui.styles.yt_base_styles.yt.base.styles.css.js,third_party.javascript.youtube_components.tp_yt_paper_ripple.tp.yt.paper.ripple.css.js--><div id="background" class="style-scope tp-yt-paper-ripple"></div>
<div id="waves" class="style-scope tp-yt-paper-ripple"></div>
</tp-yt-paper-ripple></tp-yt-paper-button>

In [19]:
## 
empty = ""
empty + 'a'

'a'

In [ ]:
<tr class="description-item style-scope ytd-about-channel-renderer">
        <td class="style-scope ytd-about-channel-renderer">
          <yt-icon icon="info_outline" class="style-scope ytd-about-channel-renderer"><!--css-build:shady--><!--css_build_scope:yt-icon--><!--css_build_styles:video.youtube.src.web.polymer.shared.ui.styles.yt_base_styles.yt.base.styles.css.js,video.youtube.src.web.polymer.shared.core.yt_icon.yt.icon.css.js--><span class="yt-icon-shape style-scope yt-icon ytSpecIconShapeHost"><div style="width: 100%; height: 100%; display: block; fill: currentcolor;"><svg xmlns="http://www.w3.org/2000/svg" height="24" viewBox="0 0 24 24" width="24" focusable="false" aria-hidden="true" style="pointer-events: none; display: inherit; width: 100%; height: 100%;"><path d="M13 17h-2v-6h2v6zm0-10h-2v2h2V7zm-1-4c-4.96 0-9 4.04-9 9s4.04 9 9 9 9-4.04 9-9-4.04-9-9-9m0-1c5.52 0 10 4.48 10 10s-4.48 10-10 10S2 17.52 2 12 6.48 2 12 2z"></path></svg></div></span></yt-icon>
        </td>
        <td class="style-scope ytd-about-channel-renderer">
          <yt-attributed-string class="style-scope ytd-about-channel-renderer"><span class="yt-core-attributed-string yt-core-attributed-string--white-space-pre-wrap" role="text"><span class="" style="">Joined Jan 28, 2007</span></span></yt-attributed-string>
        </td>
      </tr>

In [8]:
import os 
print(os.getcwd())

/Users/Malcolm/Desktop/Malaysian_Banking_Youtube_Wars/CAIE_PROJECT


In [ ]:
<yt-touch-feedback-shape aria-hidden="true" class="yt-spec-touch-feedback-shape yt-spec-touch-feedback-shape--touch-response"><div class="yt-spec-touch-feedback-shape__stroke"></div><div class="yt-spec-touch-feedback-shape__fill"></div></yt-touch-feedback-shape>